# Sensorless Drive Diagnosis

> In this example, the main focus is the classification of individual states of a motor.

In [ ]:
# |hide
from nbdev.showdoc import *

In [ ]:
# | hide
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import pandas as pd
import copy

from sklearn.metrics import classification_report
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from matplotlib import pyplot as plt
import seaborn as sns

from mlmvn.layers import FirstLayer, HiddenLayer, OutputLayer, cmplx_phase_activation
from mlmvn.loss import ComplexMSELoss
from mlmvn.optim import MySGD, ECL
from pathlib import Path
from clearml import Task, Logger

In [ ]:
# | hide
# --- helper functions ---
def reverse_one_hot(x, neuronCats):
    a = np.zeros(len(x))
    x = torch.detach(x)
    for i in range(len(x)):
        a[i] = torch.max(x[i]) - 1 + np.argmax(x[i]) * neuronCats
    return a


def accuracy(out, yb):
    out = out.type(torch.double)
    yb = yb.type(torch.double)
    x = 0
    for i in range(len(out)):
        x += torch.equal(out[i], yb[i])
    return x / len(out)


def prepare_data(x_train, x_valid, y_train, y_valid, neuronCats):
    # one-hot encoding
    numSamples, numFeatures = x_valid.shape
    y_valid_int = y_valid
    y2 = y_valid + 1  # auxiliary variable so that classes start at 1 and not 0
    numClasses = max(y2)
    target_ids = range(numClasses)
    no = int(np.ceil(numClasses / neuronCats))  # number of output neurons
    if no != 1:
        y_valid = torch.zeros(numSamples, no)
        for i in range(numSamples):
            k = int(np.ceil(y2[i] / neuronCats)) - 1
            c = np.mod((y2[i] - 1), neuronCats) + 1
            y_valid[i, k] = c
    numSamples, numFeatures = x_train.shape
    y_train_int = y_train
    y2 = y_train + 1  # auxiliary variable so that classes start at 1 and not 0
    if no != 1:
        y_train = torch.zeros(numSamples, no)
        for i in range(numSamples):
            k = int(np.ceil(y2[i] / neuronCats)) - 1
            c = np.mod((y2[i] - 1), neuronCats) + 1
            y_train[i, k] = c
    del y2

    # Convert numpy arrays into torch tensors
    x_train, y_train, x_valid, y_valid = map(
        torch.tensor, (x_train, y_train, x_valid, y_valid)
    )
    if y_train.size().__len__() == 1:
        y_train = torch.unsqueeze(y_train, 1)
        y_valid = torch.unsqueeze(y_valid, 1)

    # convert angles to complex numbers on unit-circle
    x_train = torch.exp(1.0j * x_train)
    x_valid = torch.exp(1.0j * x_valid)

    return x_train, x_valid, y_train, y_valid


def get_splitted_data(X, y, neuronCats):
    x_train, x_valid, y_train, y_valid = train_test_split(
        X, y, train_size=46806, random_state=42
    )
    x_train, x_valid, y_train, y_valid = prepare_data(
        x_train, x_valid, y_train, y_valid, neuronCats
    )

    return x_train, x_valid, y_train, y_valid


def get_splitted_data_by_index(X, y, neuronCats, train_index, test_index):
    x_train, x_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    x_train, x_valid, y_train, y_valid = prepare_data(
        x_train, x_valid, y_train, y_valid, neuronCats
    )
    return x_train, x_valid, y_train, y_valid


# --- Plots ---
def plot_loss(title, losses, scores):
    plt.rcParams["axes.grid"] = True
    fig, (ax1) = plt.subplots(1, 1, figsize=(8, 4))
    fig.suptitle("CVNN - Moons")
    ax1.plot(np.linspace(1, len(losses), len(losses)), losses)
    ax1.set_xlabel("Epoch")
    ax1.set_xlim(0, len(losses))

    ax1.plot(np.linspace(1, len(scores), len(scores)), scores)
    ax1.set_xlabel("Epoch")
    ax1.set_xlim(0, len(losses))

    ax1.legend(["Acc", "Loss"])

    plt.show()


def plot_weights(title, ylabel_1, ylabel_2, weights_real, weights_imag):
    # y_min = np.min([np.min(weights_real), np.min(weights_imag)])
    # y_max = np.max([np.max(weights_real), np.max(weights_imag)])

    fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(14, 3))
    fig.suptitle(title)
    ax[0].plot(np.linspace(1, len(weights_real), len(weights_real)), weights_real)
    ax[0].set_xlabel("Step")
    ax[0].set_ylabel(ylabel_1)
    # ax[0].set_title("Real Valued Weigts")
    ax[0].set_xlim(0, len(weights_real))
    # ax[0].set_ylim(y_min, y_max)

    ax[1].plot(np.linspace(1, len(weights_imag), len(weights_imag)), weights_imag)
    ax[1].set_xlabel("Step")
    ax[1].set_ylabel(ylabel_2)
    # ax[1].set_title("Imaginary Valued Weights")
    ax[1].set_xlim(0, len(weights_imag))
    # ax[1].set_ylim(y_min, y_max)

    plt.show()


def plot_loss_acc_list(title, list_losses, list_scores, image_name):
    losses = np.mean(list_losses, axis=0)
    scores = np.mean(list_scores, axis=0)

    losses_std = np.std(list_losses, axis=0)
    scores_std = np.std(list_scores, axis=0)

    fig, (ax1) = plt.subplots(1, 1, figsize=(10, 3))
    fig.suptitle(title)
    ax1.plot(np.linspace(1, len(losses), len(losses)), losses)
    ax1.fill_between(
        np.linspace(1, len(losses), len(losses)),
        losses + losses_std,
        losses - losses_std,
        alpha=0.5,
        linewidth=0,
    )

    ax1.plot(np.linspace(1, len(scores), len(scores)), scores)
    ax1.fill_between(
        np.linspace(1, len(scores), len(scores)),
        scores + scores_std,
        scores - scores_std,
        alpha=0.5,
        linewidth=0,
    )
    ax1.set_xlabel("Epoch")

    plt.legend(["Loss Mean", "Loss Std", "Acc. Mean", "Acc. Std"])
    fig.savefig(image_name, format="png", dpi=600)

    plt.show()
    # save
    # fig.savefig(image_name + ".svg", format="svg", dpi=600)


# --- Logging ---
model_dict: dict = {}


def fc_hook(layer_name, module, grad_input, grad_output):
    if layer_name in model_dict:
        model_dict[layer_name]["weights"] = module.weights.detach().clone()
        model_dict[layer_name]["bias"] = module.bias.detach().clone()
        model_dict[layer_name]["grad_input"] = grad_input
        model_dict[layer_name]["grad_output"] = grad_output
    else:
        model_dict[layer_name] = {}
        model_dict[layer_name]["weights"] = module.weights.detach().clone()
        model_dict[layer_name]["bias"] = module.bias.detach().clone()
        model_dict[layer_name]["grad_input"] = grad_input
        model_dict[layer_name]["grad_output"] = grad_output

In [ ]:
# | hide
# control variables
# number of categories a neuron can distinguish / parameter that determines the number of output neurons
neuronCats = 1
# number of categories per neuron, i.e. neuronCats (+ 1 for others in case of multiple Outputs)
categories = 2
# how often a classification sector occurs (1 means no periodicity)
periodicity = 1
# path to store best model parameters

## Load Data

In [ ]:
train_csv = pd.read_csv(
    "data/autass_data2.csv",
    header=None,
    dtype=np.double,
)
data = np.array(train_csv.values[:, 1:50])
del train_csv

In [ ]:
X = data[:, 0:48]
y = data[:, 48].astype(int) - 1

yt = copy.copy(y)
yt[yt == 0] = 20
yt[yt == 1] = 21
yt[yt == 2] = 22
yt[yt == 3] = 23
yt[yt == 4] = 26
yt[yt == 5] = 24
yt[yt == 6] = 27
yt[yt == 7] = 29
yt[yt == 8] = 30
yt[yt == 9] = 25
yt[yt == 10] = 28
yt -= 20
y = yt
del yt

## Config

In [ ]:
epochs = 200
batch_size = 538
lr = 1
clip_angle_value = 1000000

## Single Layer

### MLMVN [48-10-11]

In [ ]:
PATH = str(Path.cwd() / "models/autass-mlmvn_48-10-11.pt")

In [ ]:
class Model(nn.Module):
    def __init__(self, categories, periodicity):
        super().__init__()
        self.categories = categories
        self.periodicity = periodicity
        self.first_linear = FirstLayer(48, 10)
        self.phase_act1 = cmplx_phase_activation()
        self.linear_out = OutputLayer(10, 11)
        self.phase_act2 = cmplx_phase_activation()
        # Hooks
        self.first_layer_hook_handle = self.first_linear.register_full_backward_hook(
            self.first_layer_backward_hook
        )
        self.output_hook_handle = self.linear_out.register_full_backward_hook(
            self.output_layer_backward_hook
        )

    def forward(self, x):
        x = self.first_linear(x)
        x = self.phase_act1(x)
        x = self.linear_out(x)
        x = self.phase_act2(x)
        return x

    def first_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("first_layer", module, grad_input, grad_output)

    def hidden_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("hidden_layer", module, grad_input, grad_output)

    def output_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("output_layer", module, grad_input, grad_output)

    def angle2class(self, x: torch.tensor) -> torch.tensor:
        tmp = x.angle() + 2 * np.pi
        angle = torch.remainder(tmp, 2 * np.pi)

        # This will be the discrete output (the number of sector)
        o = torch.floor(self.categories * self.periodicity * angle / (2 * np.pi))
        return torch.remainder(o, self.categories)

    def predict(self, x):
        """
        Performs the prediction task of the network

        Args:
          x: torch.Tensor
            Input tensor of size ([3])

        Returns:
          Most likely class i.e., Label with the highest score
        """
        # Pass the data through the networks
        output = self.forward(x)

        # # Choose the label with the highest score
        # return torch.argmax(output, 1)
        return self.angle2class(output)


def fit(model, X, y, epochs, batch_size, optimizer, criterion, categories, periodicity):
    # List of losses for visualization
    losses = []
    scores = []
    acc_best = 0

    for i in range(epochs):
        # Pass the data through the network and compute the loss
        # We'll use the whole dataset during the training instead of using batches
        # in to order to keep the code simple for now.

        batch_loss = []

        for j in range((X.shape[0] - 1) // batch_size + 1):
            start_j = j * batch_size
            end_j = start_j + batch_size
            xb = X[start_j:end_j]
            yb = y[start_j:end_j]

            y_pred = model(xb)
            loss = criterion(y_pred, yb, categories, periodicity)
            batch_loss.append((torch.abs(loss)).detach().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step(inputs=xb, layers=list(model.children()))

        losses.append(sum(batch_loss) / len(batch_loss))
        if i % 10 == 9:
            print(f"Epoch {i} loss is {losses[-1]}")
        y_pred = model.predict(X)
        scores.append(accuracy(y_pred.squeeze(), y))

        if scores[-1] > acc_best:
            acc_best = scores[-1]
            torch.save(model.state_dict(), PATH)
    return losses, scores

In [ ]:
task = Task.init(
    project_name="mlmvn",
    task_name="SDD-mlmvn-[48-10-11]",
    tags=["mlmvn", "SDD", "multiple_runs", "clip_angle_value"],
)
writer = SummaryWriter()

#  capture a dictionary of hyperparameters with config
config_dict = {
    "learning_rate": 1,
    "epochs": epochs,
    "batch_size": batch_size,
    "optim": "ECL",
    "categories": categories,
    "periodicity": periodicity,
    "layer": "[48-10-11]",
    "clip_angle_value": clip_angle_value,
}
task.connect(config_dict)

ClearML Task: created new task id=854e9bc7dc7e496ea841a13f617da653
ClearML results page: http://194.94.231.172:8080/projects/cdefd6ee85454e49be01962ad715eca0/experiments/854e9bc7dc7e496ea841a13f617da653/output/log


{'learning_rate': 1,
 'epochs': 200,
 'batch_size': 538,
 'optim': 'ECL',
 'categories': 2,
 'periodicity': 1,
 'layer': '[48-10-11]',
 'clip_angle_value': 1000000}

In [ ]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
list_losses = []
list_scores = []
list_acc = []
list_loss = []
list_f1 = []
list_precision = []
list_recall = []

for train_index, test_index in sss.split(X, y):
    model_dict: dict = {}
    x_train, x_valid, y_train, y_valid = get_splitted_data_by_index(
        X, y, neuronCats, train_index, test_index
    )

    model = Model(categories=categories, periodicity=periodicity)
    criterion = ComplexMSELoss.apply
    optimizer = ECL(model.parameters(), lr=lr, clip_angle_value=clip_angle_value)

    losses, scores = fit(
        model,
        x_train,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        optimizer=optimizer,
        criterion=criterion,
        categories=categories,
        periodicity=periodicity,
    )

    model.load_state_dict(torch.load(PATH))

    list_scores.append(scores)
    list_losses.append(losses)

    y_pred = model.predict(x_train)
    acc = accuracy(y_pred.squeeze(), y_train)
    print("Train Acc.: ", acc)

    y_pred = model.predict(x_valid)
    acc = accuracy(y_pred.squeeze(), y_valid)
    list_acc.append(acc)

    print(classification_report(y_valid, y_pred.detach().numpy(), zero_division=0))
    list_f1.append(
        f1_score(y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0)
    )
    list_precision.append(
        precision_score(
            y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0
        )
    )
    list_recall.append(
        recall_score(
            y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0
        )
    )

Logger.current_logger().report_single_value(
    name="val_accuracy_mean",
    value=np.mean(list_acc),
)
Logger.current_logger().report_single_value(
    name="val_accuracy_std",
    value=np.std(list_acc),
)
Logger.current_logger().report_single_value(
    name="val_f1_mean",
    value=np.mean(list_f1),
)
Logger.current_logger().report_single_value(
    name="val_f1_std",
    value=np.std(list_f1),
)
Logger.current_logger().report_single_value(
    name="val_precision_mean",
    value=np.mean(list_precision),
)
Logger.current_logger().report_single_value(
    name="val_precision_std",
    value=np.std(list_precision),
)
Logger.current_logger().report_single_value(
    name="val_recall_mean",
    value=np.mean(list_recall),
)
Logger.current_logger().report_single_value(
    name="val_recall_std",
    value=np.std(list_recall),
)

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



2022-09-27 14:25:57,872 - clearml.frameworks - INFO - Found existing registered model id=caa96da5a415490ca1ea0f95b383f403 [/home/antonpfeifer/Documents/mlmvn/nbs/examples/autass/models/autass-mlmvn_48-10-11.pt] reusing it.
Epoch 9 loss is 0.312786317016275
Epoch 19 loss is 0.25568547822538934
Epoch 29 loss is 0.3389669520974066
Epoch 39 loss is 0.28586026849296975
Epoch 49 loss is 0.2642590965695329
Epoch 59 loss is 0.2561917205608428
Epoch 69 loss is 0.2538907377290161
Epoch 79 loss is 0.25494624541893784
Epoch 89 loss is 0.2586601196421547
Epoch 99 loss is 0.26136219948247824
Epoch 109 loss is 0.25920406138033625
Epoch 119 loss is 0.257771638513778
Epoch 129 loss is 0.256717965344817
Epoch 139 loss is 0.2562337465502512
Epoch 149 loss is 0.25599395140123643
Epoch 159 loss is 0.2555283782128066
Epoch 169 loss is 0.25500644305516845
Epoch 179 loss is 0.2546311936985765
Epoch 189 loss is 0.2545335270219525
Epoch 199 loss is 0.2544423936517196
Train Acc.:  0.7605486358877945
            

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.23875833944773336
Epoch 19 loss is 0.35046091409537855
Epoch 29 loss is 0.3474688264639837
Epoch 39 loss is 0.3460845462685393
Epoch 49 loss is 0.34521615010787904
Epoch 59 loss is 0.34480239729514767
Epoch 69 loss is 0.3445531601110381
Epoch 79 loss is 0.344414452281505
Epoch 89 loss is 0.3443463357528307
Epoch 99 loss is 0.34433482782616937
Epoch 109 loss is 0.3443202594365852
Epoch 119 loss is 0.3443419606632198
Epoch 129 loss is 0.34439574951313695
Epoch 139 loss is 0.34447653188306476
Epoch 149 loss is 0.34455745559313744
Epoch 159 loss is 0.3446490512911633
Epoch 169 loss is 0.34472025490019265
Epoch 179 loss is 0.34483666034343385
Epoch 189 loss is 0.3449189385351294
Epoch 199 loss is 0.34502282438946064
Train Acc.:  0.7681329715640823
              precision    recall  f1-score   support

           0       0.84      0.86      0.85      1063
           1       0.88      0.68      0.77      1064
           2       0.90      0.73      0.81      1064
           3

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.2971094108173737
Epoch 19 loss is 0.3869108816673947
Epoch 29 loss is 0.4365067504356039
Epoch 39 loss is 0.4177013765514301
Epoch 49 loss is 0.4124413908694452
Epoch 59 loss is 0.40500117037391986
Epoch 69 loss is 0.41131196047303786
Epoch 79 loss is 0.4181249931351551
Epoch 89 loss is 0.41661642708626356
Epoch 99 loss is 0.4181854146942287
Epoch 109 loss is 0.4133348665799685
Epoch 119 loss is 0.4099116159030149
Epoch 129 loss is 0.4199283736599332
Epoch 139 loss is 0.417771504949945
Epoch 149 loss is 0.4188665644250296
Epoch 159 loss is 0.42150588067988737
Epoch 169 loss is 0.4230460352811647
Epoch 179 loss is 0.4250528931966397
Epoch 189 loss is 0.4252225452870114
Epoch 199 loss is 0.42203162628683427
Train Acc.:  0.7921464738180187
              precision    recall  f1-score   support

           0       0.95      0.88      0.92      1064
           1       0.85      0.66      0.75      1064
           2       0.92      0.80      0.86      1064
           3      

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.2795195632281697
Epoch 19 loss is 0.3375428780831629
Epoch 29 loss is 0.33256474318305307
Epoch 39 loss is 0.33185831863511417
Epoch 49 loss is 0.3295307025439074
Epoch 59 loss is 0.3210891922650079
Epoch 69 loss is 0.32034934104938245
Epoch 79 loss is 0.3163362890807787
Epoch 89 loss is 0.315736489078309
Epoch 99 loss is 0.3155383143134653
Epoch 109 loss is 0.31553327463043673
Epoch 119 loss is 0.31600845927522986
Epoch 129 loss is 0.3150536779122853
Epoch 139 loss is 0.3150594287707738
Epoch 149 loss is 0.31524582921067934
Epoch 159 loss is 0.3153440366348357
Epoch 169 loss is 0.31513214139197515
Epoch 179 loss is 0.3152703075228755
Epoch 189 loss is 0.3215461540110862
Epoch 199 loss is 0.31620214863132784
Train Acc.:  0.7109833999188155
              precision    recall  f1-score   support

           0       0.87      0.95      0.91      1063
           1       0.84      0.76      0.80      1064
           2       0.82      0.71      0.76      1064
           3   

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.39677247203032356
Epoch 19 loss is 0.35580593764225016
Epoch 29 loss is 0.42506711288150867
Epoch 39 loss is 0.4258715088317278
Epoch 49 loss is 0.42243423247066575
Epoch 59 loss is 0.4237868170495762
Epoch 69 loss is 0.41925568150605863
Epoch 79 loss is 0.439859722214063
Epoch 89 loss is 0.4537169174860011
Epoch 99 loss is 0.4516644332667944
Epoch 109 loss is 0.4505254539904648
Epoch 119 loss is 0.44988270158956056
Epoch 129 loss is 0.4494370519612547
Epoch 139 loss is 0.449528904207753
Epoch 149 loss is 0.44943834853116904
Epoch 159 loss is 0.449371091649806
Epoch 169 loss is 0.4491654165153552
Epoch 179 loss is 0.4491931206111527
Epoch 189 loss is 0.44876226191301677
Epoch 199 loss is 0.44890430281994864
Train Acc.:  0.6543465720939176
              precision    recall  f1-score   support

           0       0.70      0.86      0.77      1064
           1       0.00      0.00      0.00      1064
           2       0.95      0.68      0.79      1064
           3    

In [ ]:
task.mark_completed()
task.close()

### MLMVN [48-20-11]

In [ ]:
PATH = str(Path.cwd() / "models/autass-mlmvn_48-20-11.pt")

In [ ]:
class Model(nn.Module):
    def __init__(self, categories, periodicity):
        super().__init__()
        self.categories = categories
        self.periodicity = periodicity
        self.first_linear = FirstLayer(48, 20)
        self.phase_act1 = cmplx_phase_activation()
        self.linear_out = OutputLayer(20, 11)
        self.phase_act2 = cmplx_phase_activation()
        # Hooks
        self.first_layer_hook_handle = self.first_linear.register_full_backward_hook(
            self.first_layer_backward_hook
        )
        self.output_hook_handle = self.linear_out.register_full_backward_hook(
            self.output_layer_backward_hook
        )

    def forward(self, x):
        x = self.first_linear(x)
        x = self.phase_act1(x)
        x = self.linear_out(x)
        x = self.phase_act2(x)
        return x

    def first_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("first_layer", module, grad_input, grad_output)

    def hidden_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("hidden_layer", module, grad_input, grad_output)

    def output_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("output_layer", module, grad_input, grad_output)

    def angle2class(self, x: torch.tensor) -> torch.tensor:
        tmp = x.angle() + 2 * np.pi
        angle = torch.remainder(tmp, 2 * np.pi)

        # This will be the discrete output (the number of sector)
        o = torch.floor(self.categories * self.periodicity * angle / (2 * np.pi))
        return torch.remainder(o, self.categories)

    def predict(self, x):
        """
        Performs the prediction task of the network

        Args:
          x: torch.Tensor
            Input tensor of size ([3])

        Returns:
          Most likely class i.e., Label with the highest score
        """
        # Pass the data through the networks
        output = self.forward(x)

        # # Choose the label with the highest score
        # return torch.argmax(output, 1)
        return self.angle2class(output)


def fit(model, X, y, epochs, batch_size, optimizer, criterion, categories, periodicity):
    # List of losses for visualization
    losses = []
    scores = []
    acc_best = 0

    for i in range(epochs):
        # Pass the data through the network and compute the loss
        # We'll use the whole dataset during the training instead of using batches
        # in to order to keep the code simple for now.

        batch_loss = []

        for j in range((X.shape[0] - 1) // batch_size + 1):
            start_j = j * batch_size
            end_j = start_j + batch_size
            xb = X[start_j:end_j]
            yb = y[start_j:end_j]

            y_pred = model(xb)
            loss = criterion(y_pred, yb, categories, periodicity)
            batch_loss.append((torch.abs(loss)).detach().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step(inputs=xb, layers=list(model.children()))

        losses.append(sum(batch_loss) / len(batch_loss))
        if i % 10 == 9:
            print(f"Epoch {i} loss is {losses[-1]}")
        y_pred = model.predict(X)
        scores.append(accuracy(y_pred.squeeze(), y))

        if scores[-1] > acc_best:
            acc_best = scores[-1]
            torch.save(model.state_dict(), PATH)
    return losses, scores

In [ ]:
task = Task.init(
    project_name="mlmvn",
    task_name="SDD-mlmvn-[48-20-11]",
    tags=["mlmvn", "SDD", "multiple_runs", "clip_angle_value"],
)
writer = SummaryWriter()

#  capture a dictionary of hyperparameters with config
config_dict = {
    "learning_rate": 1,
    "epochs": epochs,
    "batch_size": batch_size,
    "optim": "ECL",
    "categories": categories,
    "periodicity": periodicity,
    "layer": "[48-20-11]",
    "clip_angle_value": clip_angle_value,
}
task.connect(config_dict)

ClearML Task: created new task id=47852b7bd30c475a9fbb69e5802aea36
ClearML results page: http://194.94.231.172:8080/projects/cdefd6ee85454e49be01962ad715eca0/experiments/47852b7bd30c475a9fbb69e5802aea36/output/log


{'learning_rate': 1,
 'epochs': 200,
 'batch_size': 538,
 'optim': 'ECL',
 'categories': 2,
 'periodicity': 1,
 'layer': '[48-20-11]',
 'clip_angle_value': 1000000}

In [ ]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
list_losses = []
list_scores = []
list_acc = []
list_loss = []
list_f1 = []
list_precision = []
list_recall = []

for train_index, test_index in sss.split(X, y):
    model_dict: dict = {}
    x_train, x_valid, y_train, y_valid = get_splitted_data_by_index(
        X, y, neuronCats, train_index, test_index
    )

    model = Model(categories=categories, periodicity=periodicity)
    criterion = ComplexMSELoss.apply
    optimizer = ECL(model.parameters(), lr=lr, clip_angle_value=clip_angle_value)

    losses, scores = fit(
        model,
        x_train,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        optimizer=optimizer,
        criterion=criterion,
        categories=categories,
        periodicity=periodicity,
    )

    model.load_state_dict(torch.load(PATH))

    list_scores.append(scores)
    list_losses.append(losses)

    y_pred = model.predict(x_train)
    acc = accuracy(y_pred.squeeze(), y_train)
    print("Train Acc.: ", acc)

    y_pred = model.predict(x_valid)
    acc = accuracy(y_pred.squeeze(), y_valid)
    list_acc.append(acc)

    print(classification_report(y_valid, y_pred.detach().numpy(), zero_division=0))
    list_f1.append(
        f1_score(y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0)
    )
    list_precision.append(
        precision_score(
            y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0
        )
    )
    list_recall.append(
        recall_score(
            y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0
        )
    )

Logger.current_logger().report_single_value(
    name="val_accuracy_mean",
    value=np.mean(list_acc),
)
Logger.current_logger().report_single_value(
    name="val_accuracy_std",
    value=np.std(list_acc),
)
Logger.current_logger().report_single_value(
    name="val_f1_mean",
    value=np.mean(list_f1),
)
Logger.current_logger().report_single_value(
    name="val_f1_std",
    value=np.std(list_f1),
)
Logger.current_logger().report_single_value(
    name="val_precision_mean",
    value=np.mean(list_precision),
)
Logger.current_logger().report_single_value(
    name="val_precision_std",
    value=np.std(list_precision),
)
Logger.current_logger().report_single_value(
    name="val_recall_mean",
    value=np.mean(list_recall),
)
Logger.current_logger().report_single_value(
    name="val_recall_std",
    value=np.std(list_recall),
)

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



2022-09-27 14:42:43,693 - clearml.frameworks - INFO - Found existing registered model id=c337b94a22444d809d449783726d8ee2 [/home/antonpfeifer/Documents/mlmvn/nbs/examples/autass/models/autass-mlmvn_48-20-11.pt] reusing it.
Epoch 9 loss is 0.15395791001477882
Epoch 19 loss is 0.15815942352420895
Epoch 29 loss is 0.1521475958209485
Epoch 39 loss is 0.22264992876002446
Epoch 49 loss is 0.2434950175800985
Epoch 59 loss is 0.3000683165649394
Epoch 69 loss is 0.2986102485389353
Epoch 79 loss is 0.2978965348099431
Epoch 89 loss is 0.2976461313968712
Epoch 99 loss is 0.297615187920047
Epoch 109 loss is 0.2975814279553468
Epoch 119 loss is 0.2973692060082367
Epoch 129 loss is 0.2971646093759705
Epoch 139 loss is 0.2969823604569539
Epoch 149 loss is 0.2968226945940804
Epoch 159 loss is 0.29669339181528953
Epoch 169 loss is 0.2965860987914452
Epoch 179 loss is 0.2964960589856526
Epoch 189 loss is 0.296397997745145
Epoch 199 loss is 0.2963285508675669
Train Acc.:  0.8742068494028671
              

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.23371587049756218
Epoch 19 loss is 0.2171053793431578
Epoch 29 loss is 0.2117481726413607
Epoch 39 loss is 0.20548662762905423
Epoch 49 loss is 0.20055165801015304
Epoch 59 loss is 0.19926006465921117
Epoch 69 loss is 0.20121967169245297
Epoch 79 loss is 0.2012524994483403
Epoch 89 loss is 0.2089158789663104
Epoch 99 loss is 0.2069847307533938
Epoch 109 loss is 0.20158804575222558
Epoch 119 loss is 0.19908199121184547
Epoch 129 loss is 0.1983263363973267
Epoch 139 loss is 0.1981246632369139
Epoch 149 loss is 0.19843244794655257
Epoch 159 loss is 0.19990613128620335
Epoch 169 loss is 0.20346991661240654
Epoch 179 loss is 0.20220646457032931
Epoch 189 loss is 0.20162040661533578
Epoch 199 loss is 0.20261586962160896
Train Acc.:  0.7864421988164163
              precision    recall  f1-score   support

           0       0.95      0.91      0.93      1063
           1       0.90      0.78      0.84      1064
           2       0.98      0.66      0.79      1064
         

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.17359070471532762
Epoch 19 loss is 0.19923888591987557
Epoch 29 loss is 0.22116873452047683
Epoch 39 loss is 0.2029670658449249
Epoch 49 loss is 0.20419508592690006
Epoch 59 loss is 0.19794629333818395
Epoch 69 loss is 0.1952743740865991
Epoch 79 loss is 0.19510413114878267
Epoch 89 loss is 0.19646997157487037
Epoch 99 loss is 0.19606822933983284
Epoch 109 loss is 0.19653955121411834
Epoch 119 loss is 0.19678779104367372
Epoch 129 loss is 0.19675274303142729
Epoch 139 loss is 0.19700162446357028
Epoch 149 loss is 0.1971595217735474
Epoch 159 loss is 0.19742537431793827
Epoch 169 loss is 0.19757406298027266
Epoch 179 loss is 0.1976568692427292
Epoch 189 loss is 0.19808705126678153
Epoch 199 loss is 0.19829968689615954
Train Acc.:  0.8086610976990621
              precision    recall  f1-score   support

           0       0.93      0.90      0.91      1064
           1       0.87      0.82      0.85      1064
           2       0.97      0.91      0.94      1064
      

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.16241861081194287
Epoch 19 loss is 0.142375697260239
Epoch 29 loss is 0.15385840834181844
Epoch 39 loss is 0.2138264016307598
Epoch 49 loss is 0.2365240586587247
Epoch 59 loss is 0.2929398953187324
Epoch 69 loss is 0.2761381170166951
Epoch 79 loss is 0.3456885180336249
Epoch 89 loss is 0.34346247018042997
Epoch 99 loss is 0.3458485532239313
Epoch 109 loss is 0.3444044925757802
Epoch 119 loss is 0.3440097919960899
Epoch 129 loss is 0.34428306482555965
Epoch 139 loss is 0.3449518206799149
Epoch 149 loss is 0.3461375016053481
Epoch 159 loss is 0.3476872522585145
Epoch 169 loss is 0.34796445004321847
Epoch 179 loss is 0.3482418109346652
Epoch 189 loss is 0.34871055688690134
Epoch 199 loss is 0.3490575960919955
Train Acc.:  0.8840558036191168
              precision    recall  f1-score   support

           0       0.94      0.94      0.94      1063
           1       0.92      0.90      0.91      1064
           2       0.97      0.93      0.95      1064
           3     

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.15193523247470855
Epoch 19 loss is 0.1408373010379282
Epoch 29 loss is 0.18472082419796915
Epoch 39 loss is 0.1959035029241411
Epoch 49 loss is 0.21619177741693082
Epoch 59 loss is 0.21511564006011144
Epoch 69 loss is 0.20399205995454375
Epoch 79 loss is 0.20022756204760384
Epoch 89 loss is 0.20033638712455112
Epoch 99 loss is 0.19893451606025606
Epoch 109 loss is 0.19786037468191467
Epoch 119 loss is 0.19890569036312059
Epoch 129 loss is 0.1990134917790664
Epoch 139 loss is 0.19872859569206974
Epoch 149 loss is 0.19751614804725498
Epoch 159 loss is 0.19739665962479105
Epoch 169 loss is 0.19784372444886006
Epoch 179 loss is 0.19111926939075988
Epoch 189 loss is 0.19073564886846614
Epoch 199 loss is 0.18976225840289762
Train Acc.:  0.8771764906958361
              precision    recall  f1-score   support

           0       0.95      0.96      0.95      1064
           1       0.89      0.79      0.84      1064
           2       0.97      0.93      0.95      1064
     

In [ ]:
task.mark_completed()
task.close()

### MLMVN [48-50-11]

In [ ]:
PATH = str(Path.cwd() / "models/autass-mlmvn_48-50-11.pt")

In [ ]:
class Model(nn.Module):
    def __init__(self, categories, periodicity):
        super().__init__()
        self.categories = categories
        self.periodicity = periodicity
        self.first_linear = FirstLayer(48, 50)
        self.phase_act1 = cmplx_phase_activation()
        self.linear_out = OutputLayer(50, 11)
        self.phase_act2 = cmplx_phase_activation()
        # Hooks
        self.first_layer_hook_handle = self.first_linear.register_full_backward_hook(
            self.first_layer_backward_hook
        )
        self.output_hook_handle = self.linear_out.register_full_backward_hook(
            self.output_layer_backward_hook
        )

    def forward(self, x):
        x = self.first_linear(x)
        x = self.phase_act1(x)
        x = self.linear_out(x)
        x = self.phase_act2(x)
        return x

    def first_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("first_layer", module, grad_input, grad_output)

    def hidden_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("hidden_layer", module, grad_input, grad_output)

    def output_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("output_layer", module, grad_input, grad_output)

    def angle2class(self, x: torch.tensor) -> torch.tensor:
        tmp = x.angle() + 2 * np.pi
        angle = torch.remainder(tmp, 2 * np.pi)

        # This will be the discrete output (the number of sector)
        o = torch.floor(self.categories * self.periodicity * angle / (2 * np.pi))
        return torch.remainder(o, self.categories)

    def predict(self, x):
        """
        Performs the prediction task of the network

        Args:
          x: torch.Tensor
            Input tensor of size ([3])

        Returns:
          Most likely class i.e., Label with the highest score
        """
        # Pass the data through the networks
        output = self.forward(x)

        # # Choose the label with the highest score
        # return torch.argmax(output, 1)
        return self.angle2class(output)


def fit(model, X, y, epochs, batch_size, optimizer, criterion, categories, periodicity):
    # List of losses for visualization
    losses = []
    scores = []
    acc_best = 0

    for i in range(epochs):
        # Pass the data through the network and compute the loss
        # We'll use the whole dataset during the training instead of using batches
        # in to order to keep the code simple for now.

        batch_loss = []

        for j in range((X.shape[0] - 1) // batch_size + 1):
            start_j = j * batch_size
            end_j = start_j + batch_size
            xb = X[start_j:end_j]
            yb = y[start_j:end_j]

            y_pred = model(xb)
            loss = criterion(y_pred, yb, categories, periodicity)
            batch_loss.append((torch.abs(loss)).detach().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step(inputs=xb, layers=list(model.children()))

        losses.append(sum(batch_loss) / len(batch_loss))
        if i % 10 == 9:
            print(f"Epoch {i} loss is {losses[-1]}")
        y_pred = model.predict(X)
        scores.append(accuracy(y_pred.squeeze(), y))

        if scores[-1] > acc_best:
            acc_best = scores[-1]
            torch.save(model.state_dict(), PATH)
    return losses, scores

In [ ]:
task = Task.init(
    project_name="mlmvn",
    task_name="SDD-mlmvn-[48-50-11]",
    tags=["mlmvn", "SDD", "multiple_runs", "clip_angle_value"],
)
writer = SummaryWriter()

#  capture a dictionary of hyperparameters with config
config_dict = {
    "learning_rate": 1,
    "epochs": epochs,
    "batch_size": batch_size,
    "optim": "ECL",
    "categories": categories,
    "periodicity": periodicity,
    "layer": "[48-50-11]",
    "clip_angle_value": clip_angle_value,
}
task.connect(config_dict)

ClearML Task: created new task id=0e2885b9c6934f79849153fe8c7149d5
ClearML results page: http://194.94.231.172:8080/projects/cdefd6ee85454e49be01962ad715eca0/experiments/0e2885b9c6934f79849153fe8c7149d5/output/log


{'learning_rate': 1,
 'epochs': 200,
 'batch_size': 538,
 'optim': 'ECL',
 'categories': 2,
 'periodicity': 1,
 'layer': '[48-50-11]',
 'clip_angle_value': 1000000}

In [ ]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
list_losses = []
list_scores = []
list_acc = []
list_loss = []
list_f1 = []
list_precision = []
list_recall = []

for train_index, test_index in sss.split(X, y):
    model_dict: dict = {}
    x_train, x_valid, y_train, y_valid = get_splitted_data_by_index(
        X, y, neuronCats, train_index, test_index
    )

    model = Model(categories=categories, periodicity=periodicity)
    criterion = ComplexMSELoss.apply
    optimizer = ECL(model.parameters(), lr=lr, clip_angle_value=clip_angle_value)

    losses, scores = fit(
        model,
        x_train,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        optimizer=optimizer,
        criterion=criterion,
        categories=categories,
        periodicity=periodicity,
    )

    model.load_state_dict(torch.load(PATH))

    list_scores.append(scores)
    list_losses.append(losses)

    y_pred = model.predict(x_train)
    acc = accuracy(y_pred.squeeze(), y_train)
    print("Train Acc.: ", acc)

    y_pred = model.predict(x_valid)
    acc = accuracy(y_pred.squeeze(), y_valid)
    list_acc.append(acc)

    print(classification_report(y_valid, y_pred.detach().numpy(), zero_division=0))
    list_f1.append(
        f1_score(y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0)
    )
    list_precision.append(
        precision_score(
            y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0
        )
    )
    list_recall.append(
        recall_score(
            y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0
        )
    )

Logger.current_logger().report_single_value(
    name="val_accuracy_mean",
    value=np.mean(list_acc),
)
Logger.current_logger().report_single_value(
    name="val_accuracy_std",
    value=np.std(list_acc),
)
Logger.current_logger().report_single_value(
    name="val_f1_mean",
    value=np.mean(list_f1),
)
Logger.current_logger().report_single_value(
    name="val_f1_std",
    value=np.std(list_f1),
)
Logger.current_logger().report_single_value(
    name="val_precision_mean",
    value=np.mean(list_precision),
)
Logger.current_logger().report_single_value(
    name="val_precision_std",
    value=np.std(list_precision),
)
Logger.current_logger().report_single_value(
    name="val_recall_mean",
    value=np.mean(list_recall),
)
Logger.current_logger().report_single_value(
    name="val_recall_std",
    value=np.std(list_recall),
)

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



2022-09-27 14:56:58,474 - clearml.frameworks - INFO - Found existing registered model id=bb96e63090904339bf87c4852d30bdb6 [/home/antonpfeifer/Documents/mlmvn/nbs/examples/autass/models/autass-mlmvn_48-50-11.pt] reusing it.
Epoch 9 loss is 0.11677723785531834
Epoch 19 loss is 0.08727784719556735
Epoch 29 loss is 0.09157292581128637
Epoch 39 loss is 0.07421681373324716
Epoch 49 loss is 0.07697537707769428
Epoch 59 loss is 0.06723158685548768
Epoch 69 loss is 0.06324904770714807
Epoch 79 loss is 0.0635766536591303
Epoch 89 loss is 0.0689652033514836
Epoch 99 loss is 0.09650615274908997
Epoch 109 loss is 0.07453832805905608
Epoch 119 loss is 0.07929171747710538
Epoch 129 loss is 0.07981405980555767
Epoch 139 loss is 0.09314429308579714
Epoch 149 loss is 0.1007599468837455
Epoch 159 loss is 0.1594258365172202
Epoch 169 loss is 0.16438126926324473
Epoch 179 loss is 0.1535026421169871
Epoch 189 loss is 0.15247239398711698
Epoch 199 loss is 0.1501105166766776
Train Acc.:  0.9464609994231632
  

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.10269231647718571
Epoch 19 loss is 0.0828424365293163
Epoch 29 loss is 0.0755315495091804
Epoch 39 loss is 0.07214960396708685
Epoch 49 loss is 0.06101909084585404
Epoch 59 loss is 0.06839280821097835
Epoch 69 loss is 0.0599125453507835
Epoch 79 loss is 0.07335060013207284
Epoch 89 loss is 0.07783843407293703
Epoch 99 loss is 0.07579400460243982
Epoch 109 loss is 0.08774453313883374
Epoch 119 loss is 0.07845111177806056
Epoch 129 loss is 0.08304869556239905
Epoch 139 loss is 0.09663094311768118
Epoch 149 loss is 0.09574537896135903
Epoch 159 loss is 0.10745339520496501
Epoch 169 loss is 0.12221249713581007
Epoch 179 loss is 0.1238504916702643
Epoch 189 loss is 0.12257097729943099
Epoch 199 loss is 0.12221594344523189
Train Acc.:  0.9447732176811161
              precision    recall  f1-score   support

           0       0.98      0.97      0.98      1063
           1       0.96      0.94      0.95      1064
           2       0.98      0.98      0.98      1064
      

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.11407119840787741
Epoch 19 loss is 0.0782730350170469
Epoch 29 loss is 0.07717022981852589
Epoch 39 loss is 0.07464963948878549
Epoch 49 loss is 0.07757185254979934
Epoch 59 loss is 0.06594024636591442
Epoch 69 loss is 0.06721012903498128
Epoch 79 loss is 0.06679680268106901
Epoch 89 loss is 0.06374956514109469
Epoch 99 loss is 0.06762114545661016
Epoch 109 loss is 0.06522387791331746
Epoch 119 loss is 0.0730944025786924
Epoch 129 loss is 0.07782279538990577
Epoch 139 loss is 0.08467630408643206
Epoch 149 loss is 0.09103032341738909
Epoch 159 loss is 0.09966637200152119
Epoch 169 loss is 0.09320824764012735
Epoch 179 loss is 0.12129874941876952
Epoch 189 loss is 0.14117765266074325
Epoch 199 loss is 0.1418275397130545
Train Acc.:  0.9381716409938684
              precision    recall  f1-score   support

           0       0.97      0.96      0.96      1064
           1       0.94      0.92      0.93      1064
           2       0.99      0.96      0.98      1064
     

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.11413860654953449
Epoch 19 loss is 0.09837500329198573
Epoch 29 loss is 0.09358380064606914
Epoch 39 loss is 0.09880923285039081
Epoch 49 loss is 0.0952057750942828
Epoch 59 loss is 0.08008669991948515
Epoch 69 loss is 0.08619386331155282
Epoch 79 loss is 0.07746320343579995
Epoch 89 loss is 0.0841977986411612
Epoch 99 loss is 0.08913268603148011
Epoch 109 loss is 0.08841460231699025
Epoch 119 loss is 0.08445271805732564
Epoch 129 loss is 0.09042461018240787
Epoch 139 loss is 0.08969012531848465
Epoch 149 loss is 0.10162866889312593
Epoch 159 loss is 0.09608768953639517
Epoch 169 loss is 0.11210143946301826
Epoch 179 loss is 0.11975461444394792
Epoch 189 loss is 0.11591744085113626
Epoch 199 loss is 0.1207325082889982
Train Acc.:  0.92321661289978
              precision    recall  f1-score   support

           0       0.98      0.95      0.96      1063
           1       0.93      0.93      0.93      1064
           2       0.99      0.99      0.99      1064
       

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.10585059616812989
Epoch 19 loss is 0.08375488261089946
Epoch 29 loss is 0.0842819689311398
Epoch 39 loss is 0.07327244133203635
Epoch 49 loss is 0.06713747510860289
Epoch 59 loss is 0.06443358422771639
Epoch 69 loss is 0.061379027698900954
Epoch 79 loss is 0.06024192039466105
Epoch 89 loss is 0.06170128209518335
Epoch 99 loss is 0.05731691560047303
Epoch 109 loss is 0.06712788845175123
Epoch 119 loss is 0.06511834431841233
Epoch 129 loss is 0.06352879035121482
Epoch 139 loss is 0.06082366798673906
Epoch 149 loss is 0.07462425566574347
Epoch 159 loss is 0.07087942434442313
Epoch 169 loss is 0.09351090855570425
Epoch 179 loss is 0.08463663381504655
Epoch 189 loss is 0.09110312651864295
Epoch 199 loss is 0.09962031710031462
Train Acc.:  0.951139786784028
              precision    recall  f1-score   support

           0       0.97      0.97      0.97      1064
           1       0.96      0.91      0.93      1064
           2       0.98      0.98      0.98      1064
   

In [ ]:
task.mark_completed()
task.close()

### MLMVN [48-100-11]

In [ ]:
PATH = str(Path.cwd() / "models/autass-mlmvn_48-100-11.pt")

In [ ]:
class Model(nn.Module):
    def __init__(self, categories, periodicity):
        super().__init__()
        self.categories = categories
        self.periodicity = periodicity
        self.first_linear = FirstLayer(48, 100)
        self.phase_act1 = cmplx_phase_activation()
        self.linear_out = OutputLayer(100, 11)
        self.phase_act2 = cmplx_phase_activation()
        # Hooks
        self.first_layer_hook_handle = self.first_linear.register_full_backward_hook(
            self.first_layer_backward_hook
        )
        self.output_hook_handle = self.linear_out.register_full_backward_hook(
            self.output_layer_backward_hook
        )

    def forward(self, x):
        x = self.first_linear(x)
        x = self.phase_act1(x)
        x = self.linear_out(x)
        x = self.phase_act2(x)
        return x

    def first_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("first_layer", module, grad_input, grad_output)

    def hidden_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("hidden_layer", module, grad_input, grad_output)

    def output_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("output_layer", module, grad_input, grad_output)

    def angle2class(self, x: torch.tensor) -> torch.tensor:
        tmp = x.angle() + 2 * np.pi
        angle = torch.remainder(tmp, 2 * np.pi)

        # This will be the discrete output (the number of sector)
        o = torch.floor(self.categories * self.periodicity * angle / (2 * np.pi))
        return torch.remainder(o, self.categories)

    def predict(self, x):
        """
        Performs the prediction task of the network

        Args:
          x: torch.Tensor
            Input tensor of size ([3])

        Returns:
          Most likely class i.e., Label with the highest score
        """
        # Pass the data through the networks
        output = self.forward(x)

        # # Choose the label with the highest score
        # return torch.argmax(output, 1)
        return self.angle2class(output)


def fit(model, X, y, epochs, batch_size, optimizer, criterion, categories, periodicity):
    # List of losses for visualization
    losses = []
    scores = []
    acc_best = 0

    for i in range(epochs):
        # Pass the data through the network and compute the loss
        # We'll use the whole dataset during the training instead of using batches
        # in to order to keep the code simple for now.

        batch_loss = []

        for j in range((X.shape[0] - 1) // batch_size + 1):
            start_j = j * batch_size
            end_j = start_j + batch_size
            xb = X[start_j:end_j]
            yb = y[start_j:end_j]

            y_pred = model(xb)
            loss = criterion(y_pred, yb, categories, periodicity)
            batch_loss.append((torch.abs(loss)).detach().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step(inputs=xb, layers=list(model.children()))

        losses.append(sum(batch_loss) / len(batch_loss))
        if i % 10 == 9:
            print(f"Epoch {i} loss is {losses[-1]}")
        y_pred = model.predict(X)
        scores.append(accuracy(y_pred.squeeze(), y))

        if scores[-1] > acc_best:
            acc_best = scores[-1]
            torch.save(model.state_dict(), PATH)
    return losses, scores

In [ ]:
task = Task.init(
    project_name="mlmvn",
    task_name="SDD-mlmvn-[48-100-11]",
    tags=["mlmvn", "SDD", "multiple_runs", "clip_angle_value"],
)
writer = SummaryWriter()

#  capture a dictionary of hyperparameters with config
config_dict = {
    "learning_rate": 1,
    "epochs": epochs,
    "batch_size": batch_size,
    "optim": "ECL",
    "categories": categories,
    "periodicity": periodicity,
    "layer": "[48-100-11]",
    "clip_angle_value": clip_angle_value,
}
task.connect(config_dict)

ClearML Task: created new task id=02f388d5c7e34bd6b473b117ea6509ff
ClearML results page: http://194.94.231.172:8080/projects/cdefd6ee85454e49be01962ad715eca0/experiments/02f388d5c7e34bd6b473b117ea6509ff/output/log


{'learning_rate': 1,
 'epochs': 200,
 'batch_size': 538,
 'optim': 'ECL',
 'categories': 2,
 'periodicity': 1,
 'layer': '[48-100-11]',
 'clip_angle_value': 1000000}

In [ ]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
list_losses = []
list_scores = []
list_acc = []
list_loss = []
list_f1 = []
list_precision = []
list_recall = []

for train_index, test_index in sss.split(X, y):
    model_dict: dict = {}
    x_train, x_valid, y_train, y_valid = get_splitted_data_by_index(
        X, y, neuronCats, train_index, test_index
    )

    model = Model(categories=categories, periodicity=periodicity)
    criterion = ComplexMSELoss.apply
    optimizer = ECL(model.parameters(), lr=lr, clip_angle_value=clip_angle_value)

    losses, scores = fit(
        model,
        x_train,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        optimizer=optimizer,
        criterion=criterion,
        categories=categories,
        periodicity=periodicity,
    )

    model.load_state_dict(torch.load(PATH))

    list_scores.append(scores)
    list_losses.append(losses)

    y_pred = model.predict(x_train)
    acc = accuracy(y_pred.squeeze(), y_train)
    print("Train Acc.: ", acc)

    y_pred = model.predict(x_valid)
    acc = accuracy(y_pred.squeeze(), y_valid)
    list_acc.append(acc)

    print(classification_report(y_valid, y_pred.detach().numpy(), zero_division=0))
    list_f1.append(
        f1_score(y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0)
    )
    list_precision.append(
        precision_score(
            y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0
        )
    )
    list_recall.append(
        recall_score(
            y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0
        )
    )

Logger.current_logger().report_single_value(
    name="val_accuracy_mean",
    value=np.mean(list_acc),
)
Logger.current_logger().report_single_value(
    name="val_accuracy_std",
    value=np.std(list_acc),
)
Logger.current_logger().report_single_value(
    name="val_f1_mean",
    value=np.mean(list_f1),
)
Logger.current_logger().report_single_value(
    name="val_f1_std",
    value=np.std(list_f1),
)
Logger.current_logger().report_single_value(
    name="val_precision_mean",
    value=np.mean(list_precision),
)
Logger.current_logger().report_single_value(
    name="val_precision_std",
    value=np.std(list_precision),
)
Logger.current_logger().report_single_value(
    name="val_recall_mean",
    value=np.mean(list_recall),
)
Logger.current_logger().report_single_value(
    name="val_recall_std",
    value=np.std(list_recall),
)

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



2022-09-27 15:16:17,894 - clearml.frameworks - INFO - Found existing registered model id=0f73e6db01fc42988672e4f44c0add5f [/home/antonpfeifer/Documents/mlmvn/nbs/examples/autass/models/autass-mlmvn_48-100-11.pt] reusing it.
Epoch 9 loss is 0.08736984969525521
Epoch 19 loss is 0.07819826650816623
Epoch 29 loss is 0.06782916854065665
Epoch 39 loss is 0.04816816897126411
Epoch 49 loss is 0.05163560574118214
Epoch 59 loss is 0.04627741363342768
Epoch 69 loss is 0.04440861685817132
Epoch 79 loss is 0.03960527436632251
Epoch 89 loss is 0.040994377833916226
Epoch 99 loss is 0.03771855142496925
Epoch 109 loss is 0.037066781398096174
Epoch 119 loss is 0.03917588771049437
Epoch 129 loss is 0.03853848459391027
Epoch 139 loss is 0.039490334296889244
Epoch 149 loss is 0.03568498748548023
Epoch 159 loss is 0.037056578648648794
Epoch 169 loss is 0.03705010800859619
Epoch 179 loss is 0.03907089468945371
Epoch 189 loss is 0.0396113535151825
Epoch 199 loss is 0.04512131274939002
Train Acc.:  0.972375926

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.10348894731392981
Epoch 19 loss is 0.06993990700555251
Epoch 29 loss is 0.06444708635591645
Epoch 39 loss is 0.06056610582579465
Epoch 49 loss is 0.04929898078856764
Epoch 59 loss is 0.05208108105377289
Epoch 69 loss is 0.055182737210477444
Epoch 79 loss is 0.053781560456890025
Epoch 89 loss is 0.041558242582498066
Epoch 99 loss is 0.04481064647571699
Epoch 109 loss is 0.04177463603363158
Epoch 119 loss is 0.03897786006229341
Epoch 129 loss is 0.042479025262170365
Epoch 139 loss is 0.04130484690661847
Epoch 149 loss is 0.043655684630288405
Epoch 159 loss is 0.044328333606525876
Epoch 169 loss is 0.04163741938029155
Epoch 179 loss is 0.03787117794917678
Epoch 189 loss is 0.04063373131211509
Epoch 199 loss is 0.045135088320266366
Train Acc.:  0.9656247997094451
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      1063
           1       0.96      0.94      0.95      1064
           2       0.99      0.98      0.99      

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.09721067817250643
Epoch 19 loss is 0.0659285953883801
Epoch 29 loss is 0.05613079095748313
Epoch 39 loss is 0.053254430131788696
Epoch 49 loss is 0.050631655931046286
Epoch 59 loss is 0.043587857389422176
Epoch 69 loss is 0.04570086010022542
Epoch 79 loss is 0.039479428673811066
Epoch 89 loss is 0.03776955057557153
Epoch 99 loss is 0.04120841258334559
Epoch 109 loss is 0.05813870002363725
Epoch 119 loss is 0.04555968065778827
Epoch 129 loss is 0.03607066082124637
Epoch 139 loss is 0.03758934199215175
Epoch 149 loss is 0.033209230217276724
Epoch 159 loss is 0.02871578698680781
Epoch 169 loss is 0.029964328920410958
Epoch 179 loss is 0.03051551401962286
Epoch 189 loss is 0.031670732550214034
Epoch 199 loss is 0.03487335515855483
Train Acc.:  0.9769478924092551
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      1064
           1       0.97      0.97      0.97      1064
           2       0.99      0.98      0.99      1

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.09506221192044154
Epoch 19 loss is 0.06567186224064278
Epoch 29 loss is 0.05404001884322595
Epoch 39 loss is 0.05249845497810033
Epoch 49 loss is 0.04073885316623995
Epoch 59 loss is 0.040838370071218995
Epoch 69 loss is 0.03615846274442404
Epoch 79 loss is 0.030689715662826193
Epoch 89 loss is 0.03348275354964435
Epoch 99 loss is 0.04012077295457501
Epoch 109 loss is 0.034105905516408
Epoch 119 loss is 0.032124319295064915
Epoch 129 loss is 0.03206602824675486
Epoch 139 loss is 0.0336496624931189
Epoch 149 loss is 0.030495164592908134
Epoch 159 loss is 0.0336733646851268
Epoch 169 loss is 0.03706312464641834
Epoch 179 loss is 0.034242405249922364
Epoch 189 loss is 0.03487617796112035
Epoch 199 loss is 0.031548788032238144
Train Acc.:  0.9773965432520777
              precision    recall  f1-score   support

           0       0.99      0.98      0.99      1063
           1       0.95      0.96      0.95      1064
           2       0.99      0.99      0.99      1064


/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.09597225538158832
Epoch 19 loss is 0.07305103076821956
Epoch 29 loss is 0.06610023701716355
Epoch 39 loss is 0.058439643653002006
Epoch 49 loss is 0.047609421255355604
Epoch 59 loss is 0.050110073609477744
Epoch 69 loss is 0.044684631060736904
Epoch 79 loss is 0.041153442952646245
Epoch 89 loss is 0.03982352230327579
Epoch 99 loss is 0.036985448655858864
Epoch 109 loss is 0.04060346388820211
Epoch 119 loss is 0.0447469210668093
Epoch 129 loss is 0.03394044590851765
Epoch 139 loss is 0.03358597226108189
Epoch 149 loss is 0.027238770870941995
Epoch 159 loss is 0.029254705460162258
Epoch 169 loss is 0.05193828537228054
Epoch 179 loss is 0.030730149318143475
Epoch 189 loss is 0.03086909795063045
Epoch 199 loss is 0.03218570903393944
Train Acc.:  0.9781229303309334
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      1064
           1       0.95      0.97      0.96      1064
           2       0.99      0.99      0.99     

In [ ]:
task.mark_completed()
task.close()

## Multi Layer

### MLMVN [48-10-10-11]

In [ ]:
PATH = str(Path.cwd() / "models/autass-mlmvn_48-10-10-11.pt")

In [ ]:
class Model(nn.Module):
    def __init__(self, categories, periodicity):
        super().__init__()
        self.categories = categories
        self.periodicity = periodicity
        self.first_linear = FirstLayer(48, 10)
        self.phase_act1 = cmplx_phase_activation()
        self.hidden_layer = HiddenLayer(10, 10)
        self.phase_act2 = cmplx_phase_activation()
        self.linear_out = OutputLayer(10, 11)
        self.phase_act3 = cmplx_phase_activation()
        # Hooks
        self.first_layer_hook_handle = self.first_linear.register_full_backward_hook(
            self.first_layer_backward_hook
        )
        self.hidden_layer_hook_handle = self.hidden_layer.register_full_backward_hook(
            self.hidden_layer_backward_hook
        )
        self.output_hook_handle = self.linear_out.register_full_backward_hook(
            self.output_layer_backward_hook
        )

    def forward(self, x):
        x = self.first_linear(x)
        x = self.phase_act1(x)
        x = self.hidden_layer(x)
        x = self.phase_act2(x)
        x = self.linear_out(x)
        x = self.phase_act3(x)
        return x

    def first_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("first_layer", module, grad_input, grad_output)

    def hidden_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("hidden_layer", module, grad_input, grad_output)

    def output_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("output_layer", module, grad_input, grad_output)

    def angle2class(self, x: torch.tensor) -> torch.tensor:
        tmp = x.angle() + 2 * np.pi
        angle = torch.remainder(tmp, 2 * np.pi)

        # This will be the discrete output (the number of sector)
        o = torch.floor(self.categories * self.periodicity * angle / (2 * np.pi))
        return torch.remainder(o, self.categories)

    def predict(self, x):
        """
        Performs the prediction task of the network

        Args:
          x: torch.Tensor
            Input tensor of size ([3])

        Returns:
          Most likely class i.e., Label with the highest score
        """
        # Pass the data through the networks
        output = self.forward(x)

        # # Choose the label with the highest score
        # return torch.argmax(output, 1)
        return self.angle2class(output)


def fit(model, X, y, epochs, batch_size, optimizer, criterion, categories, periodicity):
    # List of losses for visualization
    losses = []
    scores = []
    acc_best = 0

    for i in range(epochs):
        # Pass the data through the network and compute the loss
        # We'll use the whole dataset during the training instead of using batches
        # in to order to keep the code simple for now.

        batch_loss = []

        for j in range((X.shape[0] - 1) // batch_size + 1):
            start_j = j * batch_size
            end_j = start_j + batch_size
            xb = X[start_j:end_j]
            yb = y[start_j:end_j]

            y_pred = model(xb)
            loss = criterion(y_pred, yb, categories, periodicity)
            batch_loss.append((torch.abs(loss)).detach().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step(inputs=xb, layers=list(model.children()))

        losses.append(sum(batch_loss) / len(batch_loss))
        if i % 10 == 9:
            print(f"Epoch {i} loss is {losses[-1]}")
        y_pred = model.predict(X)
        scores.append(accuracy(y_pred.squeeze(), y))

        if scores[-1] > acc_best:
            acc_best = scores[-1]
            torch.save(model.state_dict(), PATH)
    return losses, scores

In [ ]:
task = Task.init(
    project_name="mlmvn",
    task_name="SDD-mlmvn-[48-10-10-11]",
    tags=["mlmvn", "SDD", "multiple_runs", "clip_angle_value"],
)
writer = SummaryWriter()

#  capture a dictionary of hyperparameters with config
config_dict = {
    "learning_rate": 1,
    "epochs": epochs,
    "batch_size": batch_size,
    "optim": "ECL",
    "categories": categories,
    "periodicity": periodicity,
    "layer": "[48-10-10-11]",
    "clip_angle_value": clip_angle_value,
}
task.connect(config_dict)

ClearML Task: created new task id=d09b15b39d9d46629a9dd600951fbd03
ClearML results page: http://194.94.231.172:8080/projects/cdefd6ee85454e49be01962ad715eca0/experiments/d09b15b39d9d46629a9dd600951fbd03/output/log


{'learning_rate': 1,
 'epochs': 200,
 'batch_size': 538,
 'optim': 'ECL',
 'categories': 2,
 'periodicity': 1,
 'layer': '[48-10-10-11]',
 'clip_angle_value': 1000000}

In [ ]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
list_losses = []
list_scores = []
list_acc = []
list_loss = []
list_f1 = []
list_precision = []
list_recall = []

for train_index, test_index in sss.split(X, y):
    model_dict: dict = {}
    x_train, x_valid, y_train, y_valid = get_splitted_data_by_index(
        X, y, neuronCats, train_index, test_index
    )

    model = Model(categories=categories, periodicity=periodicity)
    criterion = ComplexMSELoss.apply
    optimizer = ECL(model.parameters(), lr=lr, clip_angle_value=clip_angle_value)

    losses, scores = fit(
        model,
        x_train,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        optimizer=optimizer,
        criterion=criterion,
        categories=categories,
        periodicity=periodicity,
    )

    model.load_state_dict(torch.load(PATH))

    list_scores.append(scores)
    list_losses.append(losses)

    y_pred = model.predict(x_train)
    acc = accuracy(y_pred.squeeze(), y_train)
    print("Train Acc.: ", acc)

    y_pred = model.predict(x_valid)
    acc = accuracy(y_pred.squeeze(), y_valid)
    list_acc.append(acc)

    print(classification_report(y_valid, y_pred.detach().numpy(), zero_division=0))
    list_f1.append(
        f1_score(y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0)
    )
    list_precision.append(
        precision_score(
            y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0
        )
    )
    list_recall.append(
        recall_score(
            y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0
        )
    )

Logger.current_logger().report_single_value(
    name="val_accuracy_mean",
    value=np.mean(list_acc),
)
Logger.current_logger().report_single_value(
    name="val_accuracy_std",
    value=np.std(list_acc),
)
Logger.current_logger().report_single_value(
    name="val_f1_mean",
    value=np.mean(list_f1),
)
Logger.current_logger().report_single_value(
    name="val_f1_std",
    value=np.std(list_f1),
)
Logger.current_logger().report_single_value(
    name="val_precision_mean",
    value=np.mean(list_precision),
)
Logger.current_logger().report_single_value(
    name="val_precision_std",
    value=np.std(list_precision),
)
Logger.current_logger().report_single_value(
    name="val_recall_mean",
    value=np.mean(list_recall),
)
Logger.current_logger().report_single_value(
    name="val_recall_std",
    value=np.std(list_recall),
)

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



2022-09-27 15:46:41,689 - clearml.frameworks - INFO - Found existing registered model id=410edb2915b24269b7d34f2e38593dff [/home/antonpfeifer/Documents/mlmvn/nbs/examples/autass/models/autass-mlmvn_48-10-10-11.pt] reusing it.
Epoch 9 loss is 0.36936323919586267
Epoch 19 loss is 0.4417360333030594
Epoch 29 loss is 0.4639713755873071
Epoch 39 loss is 0.47806268155361914
Epoch 49 loss is 0.46751608711861314
Epoch 59 loss is 0.4455429524911341
Epoch 69 loss is 0.44326834924033726
Epoch 79 loss is 0.4069394754230474
Epoch 89 loss is 0.41501924604142476
Epoch 99 loss is 0.4060409507954031
Epoch 109 loss is 0.4149785299989848
Epoch 119 loss is 0.4198482804118088
Epoch 129 loss is 0.4451440663334436
Epoch 139 loss is 0.48754778360762985
Epoch 149 loss is 0.512868387767623
Epoch 159 loss is 0.4912468525778975
Epoch 169 loss is 0.4819266422257403
Epoch 179 loss is 0.503401190978166
Epoch 189 loss is 0.475582148562929
Epoch 199 loss is 0.47280443258388355
Train Acc.:  0.6744931313692396
         

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.3120843412146201
Epoch 19 loss is 0.338843757118815
Epoch 29 loss is 0.29489357968814983
Epoch 39 loss is 0.3142865816273397
Epoch 49 loss is 0.33635124551797807
Epoch 59 loss is 0.37067740493614026
Epoch 69 loss is 0.336927943126264
Epoch 79 loss is 0.30800579708110587
Epoch 89 loss is 0.30535664010364083
Epoch 99 loss is 0.2919258725294692
Epoch 109 loss is 0.2816264724044177
Epoch 119 loss is 0.310759231961488
Epoch 129 loss is 0.31188458260685364
Epoch 139 loss is 0.3267504583368058
Epoch 149 loss is 0.32820414042444124
Epoch 159 loss is 0.3265853992645182
Epoch 169 loss is 0.30502072256656815
Epoch 179 loss is 0.31455467220775174
Epoch 189 loss is 0.2898721081722409
Epoch 199 loss is 0.31343407820932145
Train Acc.:  0.6747922319311214
              precision    recall  f1-score   support

           0       0.91      0.87      0.89      1063
           1       0.86      0.73      0.79      1064
           2       0.92      0.57      0.70      1064
           3   

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.5730218218958721
Epoch 19 loss is 0.5197506032800867
Epoch 29 loss is 0.5308330161877794
Epoch 39 loss is 0.5457173549411902
Epoch 49 loss is 0.4674426359824832
Epoch 59 loss is 0.48413304547490904
Epoch 69 loss is 0.5169540288793292
Epoch 79 loss is 0.46700861763673146
Epoch 89 loss is 0.42243541114495425
Epoch 99 loss is 0.441465108668188
Epoch 109 loss is 0.420622401289878
Epoch 119 loss is 0.48287951270030366
Epoch 129 loss is 0.4898719847934528
Epoch 139 loss is 0.511257985603324
Epoch 149 loss is 0.5320171382337141
Epoch 159 loss is 0.482352293969556
Epoch 169 loss is 0.5216527805105815
Epoch 179 loss is 0.5349811485542204
Epoch 189 loss is 0.5128652697548396
Epoch 199 loss is 0.5348841235805307
Train Acc.:  0.4734120964812955
              precision    recall  f1-score   support

           0       0.47      0.97      0.63      1064
           1       0.52      0.46      0.49      1064
           2       0.74      0.67      0.71      1064
           3       0.7

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.4130547185457221
Epoch 19 loss is 0.28443910819247403
Epoch 29 loss is 0.2840811952740083
Epoch 39 loss is 0.3201059445456724
Epoch 49 loss is 0.28015486618309876
Epoch 59 loss is 0.31602913086881074
Epoch 69 loss is 0.27090680930739386
Epoch 79 loss is 0.2440252615249398
Epoch 89 loss is 0.2710016949262824
Epoch 99 loss is 0.27676315756716174
Epoch 109 loss is 0.26114457901684446
Epoch 119 loss is 0.3384359963182098
Epoch 129 loss is 0.3536610052551853
Epoch 139 loss is 0.32877881416825233
Epoch 149 loss is 0.3546631938786416
Epoch 159 loss is 0.3654947728190782
Epoch 169 loss is 0.34189606918074955
Epoch 179 loss is 0.34633550078607206
Epoch 189 loss is 0.3490021138615275
Epoch 199 loss is 0.3347815447852944
Train Acc.:  0.7526438353237763
              precision    recall  f1-score   support

           0       0.73      0.98      0.84      1063
           1       0.90      0.66      0.76      1064
           2       0.93      0.83      0.88      1064
           3 

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.28021247108809677
Epoch 19 loss is 0.24448510212480346
Epoch 29 loss is 0.25997805330578505
Epoch 39 loss is 0.2997642359869558
Epoch 49 loss is 0.284409333111343
Epoch 59 loss is 0.29594549671485526
Epoch 69 loss is 0.27488304189669654
Epoch 79 loss is 0.3148477242705397
Epoch 89 loss is 0.30100965217864945
Epoch 99 loss is 0.34245337392046377
Epoch 109 loss is 0.3656904773209293
Epoch 119 loss is 0.3708399227898303
Epoch 129 loss is 0.3596192533165072
Epoch 139 loss is 0.321385041998874
Epoch 149 loss is 0.3122428807215139
Epoch 159 loss is 0.33298053660486926
Epoch 169 loss is 0.2991883369131804
Epoch 179 loss is 0.31971790791167104
Epoch 189 loss is 0.299987522555066
Epoch 199 loss is 0.31500240064659873
Train Acc.:  0.7926378533125388
              precision    recall  f1-score   support

           0       0.86      0.89      0.88      1064
           1       0.87      0.85      0.86      1064
           2       0.92      0.93      0.92      1064
           3   

In [ ]:
task.mark_completed()
task.close()

### MLMVN [48-20-20-11]

In [ ]:
PATH = str(Path.cwd() / "models/autass-mlmvn_48-20-20-11.pt")

In [ ]:
class Model(nn.Module):
    def __init__(self, categories, periodicity):
        super().__init__()
        self.categories = categories
        self.periodicity = periodicity
        self.first_linear = FirstLayer(48, 20)
        self.phase_act1 = cmplx_phase_activation()
        self.hidden_layer = HiddenLayer(20, 20)
        self.phase_act2 = cmplx_phase_activation()
        self.linear_out = OutputLayer(20, 11)
        self.phase_act3 = cmplx_phase_activation()
        # Hooks
        self.first_layer_hook_handle = self.first_linear.register_full_backward_hook(
            self.first_layer_backward_hook
        )
        self.hidden_layer_hook_handle = self.hidden_layer.register_full_backward_hook(
            self.hidden_layer_backward_hook
        )
        self.output_hook_handle = self.linear_out.register_full_backward_hook(
            self.output_layer_backward_hook
        )

    def forward(self, x):
        x = self.first_linear(x)
        x = self.phase_act1(x)
        x = self.hidden_layer(x)
        x = self.phase_act2(x)
        x = self.linear_out(x)
        x = self.phase_act3(x)
        return x

    def first_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("first_layer", module, grad_input, grad_output)

    def hidden_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("hidden_layer", module, grad_input, grad_output)

    def output_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("output_layer", module, grad_input, grad_output)

    def angle2class(self, x: torch.tensor) -> torch.tensor:
        tmp = x.angle() + 2 * np.pi
        angle = torch.remainder(tmp, 2 * np.pi)

        # This will be the discrete output (the number of sector)
        o = torch.floor(self.categories * self.periodicity * angle / (2 * np.pi))
        return torch.remainder(o, self.categories)

    def predict(self, x):
        """
        Performs the prediction task of the network

        Args:
          x: torch.Tensor
            Input tensor of size ([3])

        Returns:
          Most likely class i.e., Label with the highest score
        """
        # Pass the data through the networks
        output = self.forward(x)

        # # Choose the label with the highest score
        # return torch.argmax(output, 1)
        return self.angle2class(output)


def fit(model, X, y, epochs, batch_size, optimizer, criterion, categories, periodicity):
    # List of losses for visualization
    losses = []
    scores = []
    acc_best = 0

    for i in range(epochs):
        # Pass the data through the network and compute the loss
        # We'll use the whole dataset during the training instead of using batches
        # in to order to keep the code simple for now.

        batch_loss = []

        for j in range((X.shape[0] - 1) // batch_size + 1):
            start_j = j * batch_size
            end_j = start_j + batch_size
            xb = X[start_j:end_j]
            yb = y[start_j:end_j]

            y_pred = model(xb)
            loss = criterion(y_pred, yb, categories, periodicity)
            batch_loss.append((torch.abs(loss)).detach().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step(inputs=xb, layers=list(model.children()))

        losses.append(sum(batch_loss) / len(batch_loss))
        if i % 10 == 9:
            print(f"Epoch {i} loss is {losses[-1]}")
        y_pred = model.predict(X)
        scores.append(accuracy(y_pred.squeeze(), y))

        if scores[-1] > acc_best:
            acc_best = scores[-1]
            torch.save(model.state_dict(), PATH)
    return losses, scores

In [ ]:
task = Task.init(
    project_name="mlmvn",
    task_name="SDD-mlmvn-[48-20-20-11]",
    tags=["mlmvn", "SDD", "multiple_runs", "clip_angle_value"],
)
writer = SummaryWriter()

#  capture a dictionary of hyperparameters with config
config_dict = {
    "learning_rate": 1,
    "epochs": epochs,
    "batch_size": batch_size,
    "optim": "ECL",
    "categories": categories,
    "periodicity": periodicity,
    "layer": "[48-20-20-11]",
    "clip_angle_value": clip_angle_value,
}
task.connect(config_dict)

ClearML Task: created new task id=1ffe99f9f8014fe59562e6e70ac4e4ac
ClearML results page: http://194.94.231.172:8080/projects/cdefd6ee85454e49be01962ad715eca0/experiments/1ffe99f9f8014fe59562e6e70ac4e4ac/output/log


{'learning_rate': 1,
 'epochs': 200,
 'batch_size': 538,
 'optim': 'ECL',
 'categories': 2,
 'periodicity': 1,
 'layer': '[48-20-20-11]',
 'clip_angle_value': 1000000}

In [ ]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
list_losses = []
list_scores = []
list_acc = []
list_loss = []
list_f1 = []
list_precision = []
list_recall = []

for train_index, test_index in sss.split(X, y):
    model_dict: dict = {}
    x_train, x_valid, y_train, y_valid = get_splitted_data_by_index(
        X, y, neuronCats, train_index, test_index
    )

    model = Model(categories=categories, periodicity=periodicity)
    criterion = ComplexMSELoss.apply
    optimizer = ECL(model.parameters(), lr=lr, clip_angle_value=clip_angle_value)

    losses, scores = fit(
        model,
        x_train,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        optimizer=optimizer,
        criterion=criterion,
        categories=categories,
        periodicity=periodicity,
    )

    model.load_state_dict(torch.load(PATH))

    list_scores.append(scores)
    list_losses.append(losses)

    y_pred = model.predict(x_train)
    acc = accuracy(y_pred.squeeze(), y_train)
    print("Train Acc.: ", acc)

    y_pred = model.predict(x_valid)
    acc = accuracy(y_pred.squeeze(), y_valid)
    list_acc.append(acc)

    print(classification_report(y_valid, y_pred.detach().numpy(), zero_division=0))
    list_f1.append(
        f1_score(y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0)
    )
    list_precision.append(
        precision_score(
            y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0
        )
    )
    list_recall.append(
        recall_score(
            y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0
        )
    )

Logger.current_logger().report_single_value(
    name="val_accuracy_mean",
    value=np.mean(list_acc),
)
Logger.current_logger().report_single_value(
    name="val_accuracy_std",
    value=np.std(list_acc),
)
Logger.current_logger().report_single_value(
    name="val_f1_mean",
    value=np.mean(list_f1),
)
Logger.current_logger().report_single_value(
    name="val_f1_std",
    value=np.std(list_f1),
)
Logger.current_logger().report_single_value(
    name="val_precision_mean",
    value=np.mean(list_precision),
)
Logger.current_logger().report_single_value(
    name="val_precision_std",
    value=np.std(list_precision),
)
Logger.current_logger().report_single_value(
    name="val_recall_mean",
    value=np.mean(list_recall),
)
Logger.current_logger().report_single_value(
    name="val_recall_std",
    value=np.std(list_recall),
)

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



2022-09-27 16:04:21,821 - clearml.frameworks - INFO - Found existing registered model id=22ba5a4169ed406a9e74f40200bd29a1 [/home/antonpfeifer/Documents/mlmvn/nbs/examples/autass/models/autass-mlmvn_48-20-20-11.pt] reusing it.
Epoch 9 loss is 0.30027549864319536
Epoch 19 loss is 0.2913620808041314
Epoch 29 loss is 0.29192267719386994
Epoch 39 loss is 0.29982285425986316
Epoch 49 loss is 0.2991453145584028
Epoch 59 loss is 0.2806252701793078
Epoch 69 loss is 0.29399948094396766
Epoch 79 loss is 0.2848964264304165
Epoch 89 loss is 0.2640974185080014
Epoch 99 loss is 0.2781391938018297
Epoch 109 loss is 0.29882929257076446
Epoch 119 loss is 0.2896457279584684
Epoch 129 loss is 0.30861868284987515
Epoch 139 loss is 0.30446866634588193
Epoch 149 loss is 0.3117817060575804
Epoch 159 loss is 0.2748762158827661
Epoch 169 loss is 0.2728797345000005
Epoch 179 loss is 0.2798542257381176
Epoch 189 loss is 0.282661688657906
Epoch 199 loss is 0.30204731380523975
Train Acc.:  0.7394193176234324
      

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.3553981755313085
Epoch 19 loss is 0.3506370108940286
Epoch 29 loss is 0.30082422517534047
Epoch 39 loss is 0.30063860033973894
Epoch 49 loss is 0.25131238597560024
Epoch 59 loss is 0.23531957531915973
Epoch 69 loss is 0.2380401779400625
Epoch 79 loss is 0.2335633736232218
Epoch 89 loss is 0.23700447810534878
Epoch 99 loss is 0.21088957250172105
Epoch 109 loss is 0.23098805169700698
Epoch 119 loss is 0.2506731403676874
Epoch 129 loss is 0.24334308375596303
Epoch 139 loss is 0.25569175507335257
Epoch 149 loss is 0.2508974958920864
Epoch 159 loss is 0.23111012958349472
Epoch 169 loss is 0.22934085708672713
Epoch 179 loss is 0.23431740304916213
Epoch 189 loss is 0.2142045345957332
Epoch 199 loss is 0.220009767135271
Train Acc.:  0.7865703847715085
              precision    recall  f1-score   support

           0       0.95      0.93      0.94      1063
           1       0.86      0.81      0.83      1064
           2       0.96      0.91      0.94      1064
           

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.25758739764160093
Epoch 19 loss is 0.266251132975017
Epoch 29 loss is 0.2533447710924989
Epoch 39 loss is 0.24690942194464932
Epoch 49 loss is 0.231170000858992
Epoch 59 loss is 0.22355611411128506
Epoch 69 loss is 0.21875239712216873
Epoch 79 loss is 0.22903089863599008
Epoch 89 loss is 0.2373035179400735
Epoch 99 loss is 0.23382820577734512
Epoch 109 loss is 0.20544294051074055
Epoch 119 loss is 0.20787382558825876
Epoch 129 loss is 0.20653071272151727
Epoch 139 loss is 0.22629611145400377
Epoch 149 loss is 0.2440747135803093
Epoch 159 loss is 0.22601217778289143
Epoch 169 loss is 0.2584220808089135
Epoch 179 loss is 0.27634828629385144
Epoch 189 loss is 0.2721481887397433
Epoch 199 loss is 0.2826596067736297
Train Acc.:  0.8100070502275301
              precision    recall  f1-score   support

           0       0.85      0.95      0.90      1064
           1       0.86      0.81      0.84      1064
           2       0.90      0.90      0.90      1064
           3

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.28775937804643187
Epoch 19 loss is 0.23007369599769445
Epoch 29 loss is 0.25889260081970894
Epoch 39 loss is 0.25902661986905157
Epoch 49 loss is 0.2292498016389515
Epoch 59 loss is 0.22306006992961439
Epoch 69 loss is 0.1976271504421206
Epoch 79 loss is 0.20343043020192228
Epoch 89 loss is 0.21934100686580518
Epoch 99 loss is 0.22369418242070047
Epoch 109 loss is 0.21545978081993952
Epoch 119 loss is 0.21148563517815785
Epoch 129 loss is 0.21415086358216098
Epoch 139 loss is 0.23069838988169805
Epoch 149 loss is 0.2332487083323042
Epoch 159 loss is 0.2051832755068869
Epoch 169 loss is 0.22123131840723095
Epoch 179 loss is 0.20724936091666757
Epoch 189 loss is 0.1936030315379404
Epoch 199 loss is 0.2227826737601992
Train Acc.:  0.8283376418057128
              precision    recall  f1-score   support

           0       0.89      0.91      0.90      1063
           1       0.82      0.90      0.86      1064
           2       0.95      0.92      0.93      1064
        

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.16656616157130508
Epoch 19 loss is 0.1656779172497797
Epoch 29 loss is 0.22550141041854313
Epoch 39 loss is 0.20679155828053714
Epoch 49 loss is 0.21150408955747108
Epoch 59 loss is 0.23130154015187063
Epoch 69 loss is 0.2393321476323322
Epoch 79 loss is 0.22455397225477222
Epoch 89 loss is 0.25369244062104573
Epoch 99 loss is 0.2299914278176482
Epoch 109 loss is 0.19713104532598436
Epoch 119 loss is 0.196544944395429
Epoch 129 loss is 0.17342359035545055
Epoch 139 loss is 0.18163042589732023
Epoch 149 loss is 0.20009191348732322
Epoch 159 loss is 0.18232177900944924
Epoch 169 loss is 0.1977332400942652
Epoch 179 loss is 0.1948407359503548
Epoch 189 loss is 0.2035039176614566
Epoch 199 loss is 0.1989608963906182
Train Acc.:  0.8589099920951995
              precision    recall  f1-score   support

           0       0.92      0.96      0.94      1064
           1       0.88      0.77      0.82      1064
           2       0.89      0.95      0.92      1064
           

In [ ]:
task.mark_completed()
task.close()

### MLMVN [48-50-50-11]

In [ ]:
PATH = str(Path.cwd() / "models/autass-mlmvn_48-50-50-11.pt")

In [ ]:
class Model(nn.Module):
    def __init__(self, categories, periodicity):
        super().__init__()
        self.categories = categories
        self.periodicity = periodicity
        self.first_linear = FirstLayer(48, 50)
        self.phase_act1 = cmplx_phase_activation()
        self.hidden_layer = HiddenLayer(50, 50)
        self.phase_act2 = cmplx_phase_activation()
        self.linear_out = OutputLayer(50, 11)
        self.phase_act3 = cmplx_phase_activation()
        # Hooks
        self.first_layer_hook_handle = self.first_linear.register_full_backward_hook(
            self.first_layer_backward_hook
        )
        self.hidden_layer_hook_handle = self.hidden_layer.register_full_backward_hook(
            self.hidden_layer_backward_hook
        )
        self.output_hook_handle = self.linear_out.register_full_backward_hook(
            self.output_layer_backward_hook
        )

    def forward(self, x):
        x = self.first_linear(x)
        x = self.phase_act1(x)
        x = self.hidden_layer(x)
        x = self.phase_act2(x)
        x = self.linear_out(x)
        x = self.phase_act3(x)
        return x

    def first_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("first_layer", module, grad_input, grad_output)

    def hidden_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("hidden_layer", module, grad_input, grad_output)

    def output_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("output_layer", module, grad_input, grad_output)

    def angle2class(self, x: torch.tensor) -> torch.tensor:
        tmp = x.angle() + 2 * np.pi
        angle = torch.remainder(tmp, 2 * np.pi)

        # This will be the discrete output (the number of sector)
        o = torch.floor(self.categories * self.periodicity * angle / (2 * np.pi))
        return torch.remainder(o, self.categories)

    def predict(self, x):
        """
        Performs the prediction task of the network

        Args:
          x: torch.Tensor
            Input tensor of size ([3])

        Returns:
          Most likely class i.e., Label with the highest score
        """
        # Pass the data through the networks
        output = self.forward(x)

        # # Choose the label with the highest score
        # return torch.argmax(output, 1)
        return self.angle2class(output)


def fit(model, X, y, epochs, batch_size, optimizer, criterion, categories, periodicity):
    # List of losses for visualization
    losses = []
    scores = []
    acc_best = 0

    for i in range(epochs):
        # Pass the data through the network and compute the loss
        # We'll use the whole dataset during the training instead of using batches
        # in to order to keep the code simple for now.

        batch_loss = []

        for j in range((X.shape[0] - 1) // batch_size + 1):
            start_j = j * batch_size
            end_j = start_j + batch_size
            xb = X[start_j:end_j]
            yb = y[start_j:end_j]

            y_pred = model(xb)
            loss = criterion(y_pred, yb, categories, periodicity)
            batch_loss.append((torch.abs(loss)).detach().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step(inputs=xb, layers=list(model.children()))

        losses.append(sum(batch_loss) / len(batch_loss))
        if i % 10 == 9:
            print(f"Epoch {i} loss is {losses[-1]}")
        y_pred = model.predict(X)
        scores.append(accuracy(y_pred.squeeze(), y))

        if scores[-1] > acc_best:
            acc_best = scores[-1]
            torch.save(model.state_dict(), PATH)
    return losses, scores

In [ ]:
task = Task.init(
    project_name="mlmvn",
    task_name="SDD-mlmvn-[48-50-50-11]",
    tags=["mlmvn", "SDD", "multiple_runs", "clip_angle_value"],
)
writer = SummaryWriter()

#  capture a dictionary of hyperparameters with config
config_dict = {
    "learning_rate": 1,
    "epochs": epochs,
    "batch_size": batch_size,
    "optim": "ECL",
    "categories": categories,
    "periodicity": periodicity,
    "layer": "[48-50-50-11]",
    "clip_angle_value": clip_angle_value,
}
task.connect(config_dict)

ClearML Task: created new task id=66b39d66d7654fa1ad879e1cd8e1a4ce
ClearML results page: http://194.94.231.172:8080/projects/cdefd6ee85454e49be01962ad715eca0/experiments/66b39d66d7654fa1ad879e1cd8e1a4ce/output/log


{'learning_rate': 1,
 'epochs': 200,
 'batch_size': 538,
 'optim': 'ECL',
 'categories': 2,
 'periodicity': 1,
 'layer': '[48-50-50-11]',
 'clip_angle_value': 1000000}

In [ ]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
list_losses = []
list_scores = []
list_acc = []
list_loss = []
list_f1 = []
list_precision = []
list_recall = []

for train_index, test_index in sss.split(X, y):
    model_dict: dict = {}
    x_train, x_valid, y_train, y_valid = get_splitted_data_by_index(
        X, y, neuronCats, train_index, test_index
    )

    model = Model(categories=categories, periodicity=periodicity)
    criterion = ComplexMSELoss.apply
    optimizer = ECL(model.parameters(), lr=lr, clip_angle_value=clip_angle_value)

    losses, scores = fit(
        model,
        x_train,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        optimizer=optimizer,
        criterion=criterion,
        categories=categories,
        periodicity=periodicity,
    )

    model.load_state_dict(torch.load(PATH))

    list_scores.append(scores)
    list_losses.append(losses)

    y_pred = model.predict(x_train)
    acc = accuracy(y_pred.squeeze(), y_train)
    print("Train Acc.: ", acc)

    y_pred = model.predict(x_valid)
    acc = accuracy(y_pred.squeeze(), y_valid)
    list_acc.append(acc)

    print(classification_report(y_valid, y_pred.detach().numpy(), zero_division=0))
    list_f1.append(
        f1_score(y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0)
    )
    list_precision.append(
        precision_score(
            y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0
        )
    )
    list_recall.append(
        recall_score(
            y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0
        )
    )

Logger.current_logger().report_single_value(
    name="val_accuracy_mean",
    value=np.mean(list_acc),
)
Logger.current_logger().report_single_value(
    name="val_accuracy_std",
    value=np.std(list_acc),
)
Logger.current_logger().report_single_value(
    name="val_f1_mean",
    value=np.mean(list_f1),
)
Logger.current_logger().report_single_value(
    name="val_f1_std",
    value=np.std(list_f1),
)
Logger.current_logger().report_single_value(
    name="val_precision_mean",
    value=np.mean(list_precision),
)
Logger.current_logger().report_single_value(
    name="val_precision_std",
    value=np.std(list_precision),
)
Logger.current_logger().report_single_value(
    name="val_recall_mean",
    value=np.mean(list_recall),
)
Logger.current_logger().report_single_value(
    name="val_recall_std",
    value=np.std(list_recall),
)

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



2022-09-27 16:23:01,246 - clearml.frameworks - INFO - Found existing registered model id=f13061c5d03a4e96b788becd5e54443a [/home/antonpfeifer/Documents/mlmvn/nbs/examples/autass/models/autass-mlmvn_48-50-50-11.pt] reusing it.
Epoch 9 loss is 0.1499046038395931
Epoch 19 loss is 0.1297322766785629
Epoch 29 loss is 0.13019643237124603
Epoch 39 loss is 0.12789428760818733
Epoch 49 loss is 0.11103197831648269
Epoch 59 loss is 0.11020350228682631
Epoch 69 loss is 0.11147977013764811
Epoch 79 loss is 0.10846879683535147
Epoch 89 loss is 0.12197619828830582
Epoch 99 loss is 0.11589422750814764
Epoch 109 loss is 0.12233447725951128
Epoch 119 loss is 0.11728783170849341
Epoch 129 loss is 0.11600955988839287
Epoch 139 loss is 0.10777309333314188
Epoch 149 loss is 0.11286425886149831
Epoch 159 loss is 0.10997786349122186
Epoch 169 loss is 0.10343363161044343
Epoch 179 loss is 0.10918495130351621
Epoch 189 loss is 0.10805207773255567
Epoch 199 loss is 0.1042582369224553
Train Acc.:  0.9155681842459

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.16444180091313235
Epoch 19 loss is 0.13450935301533853
Epoch 29 loss is 0.12395481547756365
Epoch 39 loss is 0.11403245044394655
Epoch 49 loss is 0.10585260064483076
Epoch 59 loss is 0.12085044614478288
Epoch 69 loss is 0.12072065235425643
Epoch 79 loss is 0.12322402368413449
Epoch 89 loss is 0.13057146826320154
Epoch 99 loss is 0.1284516260088808
Epoch 109 loss is 0.1310643245563845
Epoch 119 loss is 0.15000504618170252
Epoch 129 loss is 0.15256414701505086
Epoch 139 loss is 0.13475608804617148
Epoch 149 loss is 0.12786701654120483
Epoch 159 loss is 0.12325916810132656
Epoch 169 loss is 0.12089844310402888
Epoch 179 loss is 0.123651544885496
Epoch 189 loss is 0.12969231060336694
Epoch 199 loss is 0.15106089017416613
Train Acc.:  0.912256713739398
              precision    recall  f1-score   support

           0       0.96      0.95      0.95      1063
           1       0.91      0.86      0.88      1064
           2       0.99      0.96      0.97      1064
       

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.13140804343389728
Epoch 19 loss is 0.11568335987799334
Epoch 29 loss is 0.10454780419932769
Epoch 39 loss is 0.10599905002509148
Epoch 49 loss is 0.10790101651025631
Epoch 59 loss is 0.10838345710268503
Epoch 69 loss is 0.1039850904931613
Epoch 79 loss is 0.10940135407457686
Epoch 89 loss is 0.09866982557423697
Epoch 99 loss is 0.10053318491512914
Epoch 109 loss is 0.09320339956264681
Epoch 119 loss is 0.10104087870858815
Epoch 129 loss is 0.10796066144525626
Epoch 139 loss is 0.10673830550703585
Epoch 149 loss is 0.10363054506764556
Epoch 159 loss is 0.10652278300728972
Epoch 169 loss is 0.11524796425634735
Epoch 179 loss is 0.1187950656937112
Epoch 189 loss is 0.11531818662457277
Epoch 199 loss is 0.10933889640581886
Train Acc.:  0.916145021043861
              precision    recall  f1-score   support

           0       0.96      0.94      0.95      1064
           1       0.91      0.91      0.91      1064
           2       1.00      0.98      0.99      1064
     

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.15382963120659784
Epoch 19 loss is 0.10859517485079796
Epoch 29 loss is 0.1039033885767897
Epoch 39 loss is 0.10722984283457825
Epoch 49 loss is 0.09191558609732126
Epoch 59 loss is 0.10290934605597563
Epoch 69 loss is 0.0952635279995302
Epoch 79 loss is 0.0910337109495511
Epoch 89 loss is 0.08792022893121838
Epoch 99 loss is 0.09991178895421955
Epoch 109 loss is 0.10163613127338143
Epoch 119 loss is 0.10929553506790932
Epoch 129 loss is 0.11517537118674706
Epoch 139 loss is 0.10527693350535264
Epoch 149 loss is 0.10425300772978957
Epoch 159 loss is 0.10921202865336493
Epoch 169 loss is 0.1063591963798746
Epoch 179 loss is 0.09805513885860391
Epoch 189 loss is 0.09630969560112046
Epoch 199 loss is 0.11513547873782298
Train Acc.:  0.9290277095306257
              precision    recall  f1-score   support

           0       0.97      0.98      0.97      1063
           1       0.91      0.93      0.92      1064
           2       0.99      0.98      0.99      1064
      

/tmp/ipykernel_10107/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.13468610442686887
Epoch 19 loss is 0.11356436751635222
Epoch 29 loss is 0.10511056101321942
Epoch 39 loss is 0.1061999656253616
Epoch 49 loss is 0.10663321393686137
Epoch 59 loss is 0.10928515764258159
Epoch 69 loss is 0.11259141153569381
Epoch 79 loss is 0.09192517041510463
Epoch 89 loss is 0.09801434421544775
Epoch 99 loss is 0.1031168064577551
Epoch 109 loss is 0.09404308404121978
Epoch 119 loss is 0.09329897373453444
Epoch 129 loss is 0.09098583010381506
Epoch 139 loss is 0.08366240511713254
Epoch 149 loss is 0.07989489898176606
Epoch 159 loss is 0.0764208793328456
Epoch 169 loss is 0.0826377881570616
Epoch 179 loss is 0.08229154744932386
Epoch 189 loss is 0.09376729251789975
Epoch 199 loss is 0.09184004778612281
Train Acc.:  0.9369752387463414
              precision    recall  f1-score   support

           0       0.96      0.97      0.97      1064
           1       0.96      0.89      0.92      1064
           2       0.99      0.98      0.98      1064
      

In [ ]:
task.mark_completed()
task.close()

### MLMVN [48-100-100-11]

In [ ]:
PATH = str(Path.cwd() / "models/autass-mlmvn_48-100-100-11.pt")

In [ ]:
class Model(nn.Module):
    def __init__(self, categories, periodicity):
        super().__init__()
        self.categories = categories
        self.periodicity = periodicity
        self.first_linear = FirstLayer(48, 100)
        self.phase_act1 = cmplx_phase_activation()
        self.hidden_layer = HiddenLayer(100, 100)
        self.phase_act2 = cmplx_phase_activation()
        self.linear_out = OutputLayer(100, 11)
        self.phase_act3 = cmplx_phase_activation()
        # Hooks
        self.first_layer_hook_handle = self.first_linear.register_full_backward_hook(
            self.first_layer_backward_hook
        )
        self.hidden_layer_hook_handle = self.hidden_layer.register_full_backward_hook(
            self.hidden_layer_backward_hook
        )
        self.output_hook_handle = self.linear_out.register_full_backward_hook(
            self.output_layer_backward_hook
        )

    def forward(self, x):
        x = self.first_linear(x)
        x = self.phase_act1(x)
        x = self.hidden_layer(x)
        x = self.phase_act2(x)
        x = self.linear_out(x)
        x = self.phase_act3(x)
        return x

    def first_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("first_layer", module, grad_input, grad_output)

    def hidden_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("hidden_layer", module, grad_input, grad_output)

    def output_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("output_layer", module, grad_input, grad_output)

    def angle2class(self, x: torch.tensor) -> torch.tensor:
        tmp = x.angle() + 2 * np.pi
        angle = torch.remainder(tmp, 2 * np.pi)

        # This will be the discrete output (the number of sector)
        o = torch.floor(self.categories * self.periodicity * angle / (2 * np.pi))
        return torch.remainder(o, self.categories)

    def predict(self, x):
        """
        Performs the prediction task of the network

        Args:
          x: torch.Tensor
            Input tensor of size ([3])

        Returns:
          Most likely class i.e., Label with the highest score
        """
        # Pass the data through the networks
        output = self.forward(x)

        # # Choose the label with the highest score
        # return torch.argmax(output, 1)
        return self.angle2class(output)


def fit(model, X, y, epochs, batch_size, optimizer, criterion, categories, periodicity):
    # List of losses for visualization
    losses = []
    scores = []
    acc_best = 0

    for i in range(epochs):
        # Pass the data through the network and compute the loss
        # We'll use the whole dataset during the training instead of using batches
        # in to order to keep the code simple for now.

        batch_loss = []

        for j in range((X.shape[0] - 1) // batch_size + 1):
            start_j = j * batch_size
            end_j = start_j + batch_size
            xb = X[start_j:end_j]
            yb = y[start_j:end_j]

            y_pred = model(xb)
            loss = criterion(y_pred, yb, categories, periodicity)
            batch_loss.append((torch.abs(loss)).detach().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step(inputs=xb, layers=list(model.children()))

        losses.append(sum(batch_loss) / len(batch_loss))
        if i % 10 == 9:
            print(f"Epoch {i} loss is {losses[-1]}")
        y_pred = model.predict(X)
        scores.append(accuracy(y_pred.squeeze(), y))

        if scores[-1] > acc_best:
            acc_best = scores[-1]
            torch.save(model.state_dict(), PATH)
    return losses, scores

In [ ]:
task = Task.init(
    project_name="mlmvn",
    task_name="SDD-mlmvn-[48-100-100-11]",
    tags=["mlmvn", "SDD", "multiple_runs", "clip_angle_value"],
)
writer = SummaryWriter()

#  capture a dictionary of hyperparameters with config
config_dict = {
    "learning_rate": lr,
    "epochs": epochs,
    "batch_size": batch_size,
    "optim": "ECL",
    "categories": categories,
    "periodicity": periodicity,
    "layer": "[48-100-100-11]",
    "clip_angle_value": clip_angle_value,
}
task.connect(config_dict)

ClearML Task: created new task id=6782b5dd458348fd9dde2a4046e3d09b
ClearML results page: http://194.94.231.172:8080/projects/cdefd6ee85454e49be01962ad715eca0/experiments/6782b5dd458348fd9dde2a4046e3d09b/output/log


{'learning_rate': 1,
 'epochs': 200,
 'batch_size': 538,
 'optim': 'ECL',
 'categories': 2,
 'periodicity': 1,
 'layer': '[48-100-100-11]',
 'clip_angle_value': 1000000}

In [ ]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
list_losses = []
list_scores = []
list_acc = []
list_loss = []
list_f1 = []
list_precision = []
list_recall = []

for train_index, test_index in sss.split(X, y):
    model_dict: dict = {}
    x_train, x_valid, y_train, y_valid = get_splitted_data_by_index(
        X, y, neuronCats, train_index, test_index
    )

    model = Model(categories=categories, periodicity=periodicity)
    criterion = ComplexMSELoss.apply
    optimizer = ECL(model.parameters(), lr=lr, clip_angle_value=clip_angle_value)

    losses, scores = fit(
        model,
        x_train,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        optimizer=optimizer,
        criterion=criterion,
        categories=categories,
        periodicity=periodicity,
    )

    model.load_state_dict(torch.load(PATH))

    list_scores.append(scores)
    list_losses.append(losses)

    y_pred = model.predict(x_train)
    acc = accuracy(y_pred.squeeze(), y_train)
    print("Train Acc.: ", acc)

    y_pred = model.predict(x_valid)
    acc = accuracy(y_pred.squeeze(), y_valid)
    list_acc.append(acc)

    print(classification_report(y_valid, y_pred.detach().numpy(), zero_division=0))
    list_f1.append(
        f1_score(y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0)
    )
    list_precision.append(
        precision_score(
            y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0
        )
    )
    list_recall.append(
        recall_score(
            y_valid, y_pred.detach().numpy(), average="weighted", zero_division=0
        )
    )

Logger.current_logger().report_single_value(
    name="val_accuracy_mean",
    value=np.mean(list_acc),
)
Logger.current_logger().report_single_value(
    name="val_accuracy_std",
    value=np.std(list_acc),
)
Logger.current_logger().report_single_value(
    name="val_f1_mean",
    value=np.mean(list_f1),
)
Logger.current_logger().report_single_value(
    name="val_f1_std",
    value=np.std(list_f1),
)
Logger.current_logger().report_single_value(
    name="val_precision_mean",
    value=np.mean(list_precision),
)
Logger.current_logger().report_single_value(
    name="val_precision_std",
    value=np.std(list_precision),
)
Logger.current_logger().report_single_value(
    name="val_recall_mean",
    value=np.mean(list_recall),
)
Logger.current_logger().report_single_value(
    name="val_recall_std",
    value=np.std(list_recall),
)

/tmp/ipykernel_15152/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



2022-09-27 20:10:34,834 - clearml.frameworks - INFO - Found existing registered model id=bbd65d869dea4025af46d264d3c7bdee [/home/antonpfeifer/Documents/mlmvn/nbs/examples/autass/models/autass-mlmvn_48-100-100-11.pt] reusing it.
Epoch 9 loss is 0.13682720639443957
Epoch 19 loss is 0.10102070636348458
Epoch 29 loss is 0.08727009396955031
Epoch 39 loss is 0.07671863965077798
Epoch 49 loss is 0.06848380096450109
Epoch 59 loss is 0.0635398093390423
Epoch 69 loss is 0.0628619211282882
Epoch 79 loss is 0.06982675314397956
Epoch 89 loss is 0.06102933808512002
Epoch 99 loss is 0.06000732133720239
Epoch 109 loss is 0.060890425387314366
Epoch 119 loss is 0.05301149732799192
Epoch 129 loss is 0.0522579315878903
Epoch 139 loss is 0.05275362570989821
Epoch 149 loss is 0.04787591243938674
Epoch 159 loss is 0.0456566457444692
Epoch 169 loss is 0.04486260011549171
Epoch 179 loss is 0.0447318709239749
Epoch 189 loss is 0.04379179251941935
Epoch 199 loss is 0.04489066304454168
Train Acc.:  0.965475249428

/tmp/ipykernel_15152/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.14070723384058115
Epoch 19 loss is 0.09901262946414933
Epoch 29 loss is 0.1163871561542634
Epoch 39 loss is 0.0928615501126676
Epoch 49 loss is 0.09462188473736881
Epoch 59 loss is 0.09051752364025904
Epoch 69 loss is 0.08058040002663806
Epoch 79 loss is 0.07118650187737878
Epoch 89 loss is 0.05725321689636519
Epoch 99 loss is 0.05587193962924077
Epoch 109 loss is 0.053161680819638193
Epoch 119 loss is 0.052564399732753064
Epoch 129 loss is 0.04833477461407704
Epoch 139 loss is 0.046382782052641146
Epoch 149 loss is 0.05173067099999342
Epoch 159 loss is 0.04534002150442024
Epoch 169 loss is 0.046009179389923965
Epoch 179 loss is 0.04352159747685905
Epoch 189 loss is 0.041239578438454445
Epoch 199 loss is 0.040097920362887236
Train Acc.:  0.9692994637554212
              precision    recall  f1-score   support

           0       0.98      0.97      0.97      1063
           1       0.94      0.93      0.94      1064
           2       1.00      0.99      0.99      106

/tmp/ipykernel_15152/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.1371787810722252
Epoch 19 loss is 0.09978662945393381
Epoch 29 loss is 0.08858285073706741
Epoch 39 loss is 0.0944868673915956
Epoch 49 loss is 0.0822929725506483
Epoch 59 loss is 0.06799231767158967
Epoch 69 loss is 0.06335458838516671
Epoch 79 loss is 0.06147011599416563
Epoch 89 loss is 0.06221068190118905
Epoch 99 loss is 0.05993916873581913
Epoch 109 loss is 0.054406843221859624
Epoch 119 loss is 0.045617025331166426
Epoch 129 loss is 0.046012246996210246
Epoch 139 loss is 0.044311349952736896
Epoch 149 loss is 0.04082695991627327
Epoch 159 loss is 0.039679863425516555
Epoch 169 loss is 0.040999900860178046
Epoch 179 loss is 0.04099159740351322
Epoch 189 loss is 0.03939206668245912
Epoch 199 loss is 0.039607182730482776
Train Acc.:  0.9713718033627449
              precision    recall  f1-score   support

           0       0.98      0.97      0.98      1064
           1       0.96      0.94      0.95      1064
           2       0.99      0.98      0.99      106

/tmp/ipykernel_15152/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.13303833512235225
Epoch 19 loss is 0.11102950469889049
Epoch 29 loss is 0.1033785956422809
Epoch 39 loss is 0.10035753702045348
Epoch 49 loss is 0.08026176993296501
Epoch 59 loss is 0.08006638138825455
Epoch 69 loss is 0.07315596454862489
Epoch 79 loss is 0.06695363560418462
Epoch 89 loss is 0.07812911516053243
Epoch 99 loss is 0.07286186109129712
Epoch 109 loss is 0.06823154445600114
Epoch 119 loss is 0.0748763280247656
Epoch 129 loss is 0.06759634080194282
Epoch 139 loss is 0.06537460445723998
Epoch 149 loss is 0.07214919456239974
Epoch 159 loss is 0.06770210388031259
Epoch 169 loss is 0.05968848252020946
Epoch 179 loss is 0.0640390788375198
Epoch 189 loss is 0.06062085278669487
Epoch 199 loss is 0.053366963455588425
Train Acc.:  0.9539812421219048
              precision    recall  f1-score   support

           0       0.98      0.97      0.97      1063
           1       0.93      0.89      0.91      1064
           2       0.99      0.98      0.98      1064
    

/tmp/ipykernel_15152/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.13915089234034914
Epoch 19 loss is 0.097373618826446
Epoch 29 loss is 0.08775521526507787
Epoch 39 loss is 0.07581347006628077
Epoch 49 loss is 0.06707371275141155
Epoch 59 loss is 0.07044893252214587
Epoch 69 loss is 0.06647936146342898
Epoch 79 loss is 0.06290139006666541
Epoch 89 loss is 0.07122418431326744
Epoch 99 loss is 0.06815365861695168
Epoch 109 loss is 0.07112648692035674
Epoch 119 loss is 0.06887879623935313
Epoch 129 loss is 0.065383156901932
Epoch 139 loss is 0.06278148365408386
Epoch 149 loss is 0.05971489880293905
Epoch 159 loss is 0.05826395838084222
Epoch 169 loss is 0.059517928423252736
Epoch 179 loss is 0.05508918404760044
Epoch 189 loss is 0.05508463686628998
Epoch 199 loss is 0.056739337282013
Train Acc.:  0.9532121263913517
              precision    recall  f1-score   support

           0       0.97      0.97      0.97      1064
           1       0.93      0.94      0.93      1064
           2       0.99      0.98      0.98      1064
       

In [ ]:
task.mark_completed()
task.close()